# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math
from numpy import log as ln

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
pre_fold = f'{drive}:/data/seoul_lab/pre'
an_fold = f'{drive}:/data/seoul_lab/analysis'
df_fold = f'{drive}:/data/seoul_lab/df'

# ❌Load
- 등록&제원
- 저감장치 부착이력
- 법정동코드
- KOSIS

## 등록&제원

In [5]:
# 전체 컬럼명
'''
'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', 
'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', 
'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', 
'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'
'''

"\n'차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', \n'배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', \n'차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', \n'삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'\n"

In [4]:
# about 4m 30.7s
name = '[ODS]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
smig = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '본거지법정동코드', '차종_등록정보', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형(소분류)', '연료', '배기량', '최초등록일자', '총중량'])
smig.info()

In [ ]:
smig.columns

Index(['본거지법정동코드', '차종_등록정보', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급',
       '차량말소YN', '차종유형(소분류)', '연료', '총중량', '배기량'],
      dtype='object')

In [ ]:
smig['차종유형(소분류)'].unique()

array(['소형', '경형', '중형', '대형', nan], dtype=object)

In [9]:
smig1 = smig.rename(columns={'본거지법정동코드':'법정동코드', '차종_등록정보':'차종', '차종유형(소분류)':'차종유형'})
smig1.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량'],
      dtype='object')

In [10]:
smig1['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25693532
Y     3840515
Name: count, dtype: int64

### 말소제거

In [11]:
smig2 = smig1[smig1['차량말소YN'] == 'N'].reset_index(drop=True)
smig2.shape

KeyboardInterrupt: 

## 저감장치 부착이력(중복차대제거)

In [ ]:
# about 28.3s
# columns = 저감장치종류, TDMS부착여부, 구조변경일자, 부착업체명, 부착일자, 탈거승인일, 차량번호, 차대번호, 저감장치반납여부, 저감장치구분, 삼원촉매장치반납여부
name = '[ODS]저감장치부착이력(중복차대제거)(2022.12.28)'
file_name = f'{name}.csv'
att = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '저감장치구분'])
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    809255 non-null  object
 1   저감장치구분  809255 non-null  object
dtypes: object(2)
memory usage: 12.3+ MB


In [ ]:
att['저감장치구분'].unique()

array(['엔진교체', '3종', '1종', '2종', '엔진개조', '1종+SCR', '삼원촉매'], dtype=object)

In [ ]:
att.loc[(att['저감장치구분'] == '1종') | (att['저감장치구분'] == '1종+SCR'), '저감장치부착유무'] = '유'

In [ ]:
att['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    512654
유      296601
Name: count, dtype: int64

## 법정동코드

In [ ]:
# about 5.6s
# columns = '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN' 
name = '법정동코드_2022.10.14(수정2023.05.04)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(big2_fold, file_name), usecols=['법정동코드', '시도', '시구군'])
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   46342 non-null  int64 
 1   시도      46342 non-null  object
 2   시구군     46319 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


## KOSIS(2021)

In [ ]:
# about 0.2s
name = '용도별_차종별_시군구별_자동차1일평균주행거리(2021)'
file_name = f'{name}.xlsx'
kos = pd.read_excel(os.path.join(ods_fold, file_name))
kos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


# ❌ 전처리

## 등급 수정

In [ ]:
smig2['배출가스등급'].value_counts(dropna=False)

배출가스등급
2     10960244
3      8623130
1      2771395
4      1511214
5      1154733
X       288966
T2      247483
T3      136301
T4          37
T5          29
Name: count, dtype: int64

In [ ]:
smig2['배출가스등급'].unique()

array(['5', 'X', 'T5', '4', '3', 'T4', 'T3', '2', '1', 'T2'], dtype=object)

In [ ]:
smig2.loc[smig2['배출가스등급'] == '1', '등급'] = '1'
smig2.loc[(smig2['배출가스등급'] == '2') | (smig2['배출가스등급'] == 'T2'), '등급'] = '2'
smig2.loc[(smig2['배출가스등급'] == '3') | (smig2['배출가스등급'] == 'T3'), '등급'] = '3'
smig2.loc[(smig2['배출가스등급'] == '4') | (smig2['배출가스등급'] == 'T4'), '등급'] = '4'
smig2.loc[(smig2['배출가스등급'] == '5') | (smig2['배출가스등급'] == 'T5'), '등급'] = '5'
smig2.loc[smig2['배출가스등급'] == 'X', '등급'] = 'X'

In [ ]:
smig2['등급'].value_counts(dropna=False)

등급
2    11207727
3     8759431
1     2771395
4     1511251
5     1154762
X      288966
Name: count, dtype: int64

## 연료 수정

In [ ]:
smig2['연료'].value_counts(dropna=False)

연료
휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
전기               368693
NaN              146798
경유 하이브리드          35779
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [ ]:
smig2['연료'].unique()

array(['휘발유', '경유', nan, '기타연료', '등유', '알코올', 'LPG(액화석유가스)',
       'CNG(압축천연가스)', '휘발유 하이브리드', 'LPG 하이브리드', '전기', 'LNG(액화천연가스)',
       '경유 하이브리드', 'CNG 하이브리드', '수소', '태양열'], dtype=object)

In [ ]:
# about 22.9s
smig2.loc[(smig2['연료'] == '휘발유') | (smig2['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
smig2.loc[(smig2['연료'] == '경유') | (smig2['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
smig2.loc[(smig2['연료'] == 'LPG(액화석유가스)') | (smig2['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
smig2.loc[(smig2['연료'] == 'CNG(압축천연가스)') | (smig2['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
smig2.loc[(smig2['연료'] == 'LNG(액화천연가스)') | (smig2['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
smig2.loc[smig2['연료'] == '전기', 'fuel'] = '전기'
smig2.loc[smig2['연료'] == '수소', 'fuel'] = '수소'
smig2.loc[smig2['연료'] == '태양열', 'fuel'] = '태양열'
smig2.loc[smig2['연료'] == '알코올', 'fuel'] = '알코올'
smig2.loc[smig2['연료'] == '등유', 'fuel'] = '등유'
smig2.loc[smig2['연료'] == '기타연료', 'fuel'] = '기타연료'
smig2.loc[smig2['연료'].isnull() == True, 'fuel'] = np.nan

In [ ]:
smig2['fuel'].value_counts(dropna=False)

fuel
휘발유     13272774
경유       9926769
LPG      1918540
전기        368693
NaN       146798
CNG        30765
수소         28227
기타연료         933
LNG           27
알코올            3
태양열            2
등유             1
Name: count, dtype: int64

In [ ]:
smig2.shape

(25693532, 14)

In [ ]:
smig2['fuel'].value_counts(dropna=False).sum()

25693532

## 중복 차대번호 제거
- 최초등록일자 최신인 데이터 남기기

In [ ]:
# 1,020대 감소
smig2.shape[0], len(smig2['차대번호'].unique())

(25693532, 25692512)

In [ ]:
smig2['최초등록일자'].describe()

count   25693532.00
mean    20141515.50
std        61276.12
min     19501231.00
25%     20110228.00
50%     20150817.00
75%     20190408.00
max     20221227.00
Name: 최초등록일자, dtype: float64

In [ ]:
smig2.sort_values('최초등록일자', ascending=False)['최초등록일자'].head()

25693531    20221227
25672286    20221227
25672262    20221227
25672261    20221227
25672260    20221227
Name: 최초등록일자, dtype: int64

In [ ]:
# 36.4s
smig3 = smig2.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
smig3.shape

(25692512, 14)

## 차령 정보 계산 및 데이터 추가

In [ ]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [ ]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel'],
      dtype='object')

In [ ]:
smig3['차령'] = current_yr - smig3['차량연식']
smig3['차령'].describe()

count   25692512.00
mean           8.55
std            6.21
min            0.00
25%            4.00
50%            7.00
75%           12.00
max          123.00
Name: 차령, dtype: float64

In [ ]:
smig3['차령'].isnull().sum()

0

In [ ]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령'],
      dtype='object')

## 법정동코드로 지역정보 부착
- 시군구명 앞지역명만 추출

In [ ]:
smig3.shape

(25692512, 15)

In [ ]:
smig4 = smig3.merge(code, on='법정동코드', how='left')
smig4.shape

(25692512, 17)

In [ ]:
# 25.9s
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

0     고양시 일산서구
9      창원시 의창구
15     전주시 완산구
18     용인시 기흥구
30     청주시 상당구
Name: 시구군, dtype: object

In [ ]:
smig4.loc[smig4['시구군'].str.contains('부산') == True, '시구군'].head()

Series([], Name: 시구군, dtype: object)

In [ ]:
# 58.3s
smig4['시구군'] = smig4['시구군'].str.split(' ').str[0]
smig4.loc[smig4['시구군'].str.split(' ').str.len() != 1, '시구군'].head()

24226351    NaN
25154486    NaN
25315223    NaN
25341440    NaN
25345354    NaN
Name: 시구군, dtype: object

In [ ]:
smig4['시구군'].head()

0    고양시
1    연수구
2     진구
3    제주시
4    제주시
Name: 시구군, dtype: object

## 1일평균주행거리 정보 추가
- 매칭 컬럼(등록&제원|kosis)
- 1\. 시도, 시구군, 차종 | 시도명, 시군구명, 차종
- 2\. 시도, 차종 | 시도명, 차종
- 3\. 시도, 시구군, 차종(nan) | 시도명, 시군구명, 차종(합계)
    - 차종:nan인 데이터는 kosis의 차종:합계의 일일평균주행거리로 매칭

In [ ]:
smig4.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [ ]:
kos.columns

Index(['시도명', '시군구명', '차종', '전체'], dtype='object')

In [ ]:
smig4.shape

(25692512, 17)

In [ ]:
# about 9.1s
# 시도, 시구군, 차종으로 매칭
smig5 = smig4.merge(kos, left_on=['시도', '시구군', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
smig5.shape

(25692512, 20)

In [ ]:
smig5['전체'].isnull().sum()

341950

In [ ]:
# 4.8s
smig5y = smig5.loc[smig5['전체'].isnull() == False]
smig5n = smig5.loc[smig5['전체'].isnull() == True]
smig5y.shape, smig5n.shape

((25350562, 20), (341950, 20))

In [ ]:
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [ ]:
smig5n = smig5n.drop(['시도명', '시군구명', '전체'], axis=1)
smig5n.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군'],
      dtype='object')

In [ ]:
smig5n.shape

(341950, 17)

In [ ]:
kos.shape

(1230, 4)

In [ ]:
kos.loc[kos['시군구명'] == '소계'].shape

(85, 4)

In [ ]:
# 시도, 차종으로만 매칭
smig5nm = smig5n.merge(kos.loc[kos['시군구명'] == '소계'].reset_index(drop=True), left_on=['시도', '차종'], right_on=['시도명', '차종'], how='left')
smig5nm.shape

(341950, 20)

In [ ]:
smig5nm.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [ ]:
smig5nm['전체'].isnull().sum()

14

In [ ]:
smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']]

,시도,시구군,차종
341013,대구광역시,달서구,NaN
341057,대구광역시,남구,NaN
341078,서울특별시,구로구,NaN
341326,대구광역시,서구,NaN
341353,경상북도,경주시,NaN
341367,경상남도,창원시,NaN
341451,대구광역시,달성군,NaN
341452,전라남도,순천시,NaN
341489,경상남도,진주시,NaN
341569,울산광역시,북구,NaN


In [ ]:
# 차종이 빈값인 데이터는 kosis의 차종이 합계인 데이터로 매칭
for a, b, c in smig5nm.loc[smig5nm['전체'].isnull() == True, ['시도', '시구군', '차종']].values:
    smig5nm.loc[(smig5nm['전체'].isnull() == True) & (smig5nm['시도'] == a) & ((smig5nm['시구군'] == b) & ((smig5nm['차종'].isnull() == True))), '전체'] = kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

In [ ]:
kos.loc[(kos['시도명'] == a) & (kos['시군구명'] == b) & (kos['차종'] == '합계'), '전체'].iloc[0]

35.7

In [ ]:
smig5nm['전체'].isnull().sum()

0

In [ ]:
# 6.4s
smig5 = pd.concat([smig5y, smig5nm], ignore_index=True)
smig5.shape

(25692512, 20)

In [ ]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '시도명',
       '시군구명', '전체'],
      dtype='object')

In [ ]:
# 12.1s
smig5 = smig5.rename(columns={'전체':'일일평균주행거리'})
smig5 = smig5.drop(['시도명', '시군구명'], axis=1)
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군',
       '일일평균주행거리'],
      dtype='object')

In [ ]:
smig5['일일평균주행거리'].isnull().sum()

0

## 저감장치 정보 추가

In [ ]:
att.columns

Index(['차대번호', '저감장치구분', '저감장치부착유무'], dtype='object')

In [ ]:
smig5.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군',
       '일일평균주행거리'],
      dtype='object')

In [ ]:
smig5.shape

(25692512, 18)

In [ ]:
# 40.9s
smig6 = smig5.merge(att[['차대번호', '저감장치부착유무']], on='차대번호', how='left')
smig6.shape

(25692512, 19)

In [ ]:
smig6['저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    25549321
유        143191
Name: count, dtype: int64

## [출력] 전처리 완료

In [ ]:
# 2m 47.1s
smig6.to_csv(os.path.join(pre_fold, '전처리완료_일일평균주행거리포함.csv'), index=False)

# [로드] 전처리

In [4]:
# 1m 18.3s
name = '전처리완료_일일평균주행거리포함'
file_name = f'{name}.csv'
smig6 = pd.read_csv(os.path.join(pre_fold, file_name), low_memory=False)
smig6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25692512 entries, 0 to 25692511
Data columns (total 19 columns):
 #   Column    Dtype  
---  ------    -----  
 0   법정동코드     int64  
 1   차종        object 
 2   차대번호      object 
 3   최초등록일자    int64  
 4   차량연식      int64  
 5   배출가스인증번호  object 
 6   배출가스등급    object 
 7   차량말소YN    object 
 8   차종유형      object 
 9   연료        object 
 10  총중량       float64
 11  배기량       float64
 12  등급        object 
 13  fuel      object 
 14  차령        int64  
 15  시도        object 
 16  시구군       object 
 17  일일평균주행거리  float64
 18  저감장치부착유무  object 
dtypes: float64(3), int64(4), object(12)
memory usage: 3.6+ GB


## 3등급 경유차 DPF부착여부 수정
- 유

In [5]:
smig6['등급'].unique()

array(['X', '1', '2', '3', '4', '5'], dtype=object)

In [6]:
smig6['fuel'].unique()

array(['휘발유', 'LPG', '경유', '전기', nan, 'CNG', '수소', '기타연료', 'LNG', '태양열',
       '알코올', '등유'], dtype=object)

In [7]:
smig6.loc[(smig6['등급'] == '3') & (smig6['fuel'] == '경유'), '저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
NaN    7618185
유            3
Name: count, dtype: int64

In [8]:
smig6.loc[(smig6['등급'] == '3') & (smig6['fuel'] == '경유'), '저감장치부착유무'] = '유'

In [9]:
smig6.loc[(smig6['등급'] == '3') & (smig6['fuel'] == '경유'), '저감장치부착유무'].value_counts(dropna=False)

저감장치부착유무
유    7618188
Name: count, dtype: int64

## 연료로 분류

In [10]:
# 10.2s
# 휘발유, 경유, LPG
dgl = smig6.loc[(smig6['fuel'] == '휘발유') | (smig6['fuel'] == '경유') | (smig6['fuel'] == 'LPG')].reset_index(drop=True)
dgl.shape

(25117098, 19)

In [11]:
# 4.6s
# 나머지 : 전기, 수소, 태양열, CNG, LNG, 등유, 알코올, 기타연료, nan
etc = smig6[(smig6['fuel'] != '휘발유') & (smig6['fuel'] != '경유') & (smig6['fuel'] != 'LPG')].reset_index(drop=True)
etc.shape

(575414, 19)

In [12]:
dgl.shape[0] + etc.shape[0]

25692512

# 배출량 계산

## 자동차-엔진 가열(Hot-start) 배출

### 배출량 식
$$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

### 주행거리(VKT)
- [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

In [13]:
# 2.6s
df2 = dgl.copy()

In [14]:
df2['VKT'] = df2['일일평균주행거리'] * 365

In [15]:
df2['VKT'].isnull().sum()

0

### DF(열화계수)
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
- 조건 수정(2023.04.20, 최)
    - 특수 소형 = 승용 소형
    - 승합 대형 = 화물 대형

In [16]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [17]:
DF_col = ['fuel', '차종', '차종유형', '차량연식', '차령']

In [18]:
df2[DF_col].head()

,fuel,차종,차종유형,차량연식,차령
0,휘발유,승용,중형,2023,0
1,휘발유,승용,중형,2023,0
2,휘발유,승용,대형,2023,0
3,휘발유,승용,대형,2023,0
4,휘발유,승용,대형,2023,0


In [19]:
df2[DF_col].values[0]

array(['휘발유', '승용', '중형', 2023, 0], dtype=object)

#### ❗ DF(열화계수) 코드
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1

In [20]:
# about 1m 14.5s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df2[DF_col].values):    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최이사님 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최이사님 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

100%|██████████| 25117098/25117098 [01:08<00:00, 366458.57it/s]


In [21]:
df2.shape[0]

25117098

In [22]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(25117098, 25117098, 25117098, 25117098)

In [23]:
df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list

In [24]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df2.loc[df2['차령'] < 10, DF_col + check_DF_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00


### 저감장치 부착 효율(R) 계산
- 저감장치부착유무 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

In [25]:
df2.loc[df2['저감장치부착유무'] == '유', ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6

In [26]:
df2.shape[0]

25117098

In [27]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']

In [28]:
df2[check_R_col].notnull().sum()

R_CO    7761369
R_HC    7761369
R_PM    7761369
dtype: int64

In [29]:
df2.loc[df2['저감장치부착유무'] == '유', DF_col + check_DF_col + check_R_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
26,경유,화물,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60
28,경유,화물,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60
60,경유,화물,소형,2022,1,1.00,1.00,1.00,1.00,99.50,90.00,83.60
90,경유,승용,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60
91,경유,승용,대형,2023,0,1.00,1.00,1.00,1.00,99.50,90.00,83.60


In [30]:
df2[check_R_col] = df2[check_R_col].fillna(0)

In [31]:
df2[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

### ❗ 배출계수(EFi)
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1
- 조건 수정(2023.04.20, 최이사님)
    - 특수 중형 = 승합 중형
    - 화물 중형 = 승합 중형
    - 화물 대형 = 승합 대형
    - 특수 중형 = 승용 중형

In [32]:
EFi_col = ['차종', '차종유형', 'fuel', '차량연식']
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [33]:
df2[EFi_col].values[0]

array(['승용', '중형', '휘발유', 2023], dtype=object)

In [34]:
df2['차종유형'].unique()

array(['중형', '대형', '경형', '소형', nan], dtype=object)

In [35]:
# about 5m 40s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []
EFi_NH3_list = []

# 차종, 차종유형, 연료, 연식
for car_type, car_size, fuel, car_birth in tqdm(df2[EFi_col].values):
    EFi_COm = 0
    EFi_HCm = 0
    EFi_NOxm = 0
    EFi_PM10m = 0
    EFi_PM2_5m = 0
    EFi_NH3m = 0
    for V in [35, 70, 100]:
        if fuel == '휘발유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 65:
                        EFi_CO = 59.783 * (V**-1.0007)
                    else:
                        EFi_CO = 0.0874 * V - 3.5618
                    EFi_HC = 8.6275 * (V**-1.0722)
                    EFi_NOx = 3.2294 * (V**-0.5763)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 5.1835 * (V**-1.1889)
                    EFi_NOx = 1.7525 * (V**-0.6481)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 65:
                        EFi_CO = 60.556 * (V**-1.2501)
                    else:
                        EFi_CO = -0.0006 * V + 0.5753
                    EFi_HC = 0.7446 * (V**-0.9392)
                    EFi_NOx = 0.3403 * (V**-0.5455)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.9952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 45:
                        EFi_CO = 4.5956 * (V**-0.8461)
                    else:
                        EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                    EFi_HC = 0.2662 * (V**-0.7830)
                    EFi_NOx = 0.4476 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 45:
                        EFi_CO = 4.4517 * (V**-0.8461)
                    else:
                        EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                    EFi_HC = 0.2556 * (V**-0.7830)
                    EFi_NOx = 0.4353 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 45:
                        EFi_CO = 4.3079 * (V**-0.8461)
                    else:
                        EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                    EFi_HC = 0.2449 * (V**-0.7830)
                    EFi_NOx = 0.4230 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.164 * (V**-0.8461)
                    else:
                        EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                    EFi_HC = 0.2343 * (V**-0.7830)
                    EFi_NOx = 0.4106 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 111.67 * (V**-1.1566)
                    EFi_HC = 32.017 * (V**-1.4171)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 22.356 * (V**-0.9068)
                    EFi_HC = 0.8428 * (V**-0.8829)
                    EFi_NOx = 1.2613 * (V**-0.3873)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 1.4898 * (V**-0.3837)
                    EFi_HC = 0.1738 * (V**-0.7268)
                    EFi_NOx = 0.1563 * (V**-0.2671)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최이사님 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 1986:
                    EFi_CO = 247.00 * (V**-0.6651)
                    EFi_HC = 15.953 * (V**-0.5059)
                    EFi_NOx = 3.1140 * (V**-0.2278)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1987 <= car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1991 <= car_birth <= 1999:
                    EFi_CO = 51.555 * (V**-0.9531)
                    EFi_HC = 31.816 * (V**-1.4804)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 29.921 * (V**-0.8868)
                    EFi_HC = 7.9374 * (V**-1.3041)
                    EFi_NOx = 1.8525 * (V**-0.4192)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 2.4938 * (V**-0.6106)
                    EFi_HC = 0.4262 * (V**-1.0122)
                    EFi_NOx = 0.1818 * (V**-0.4316)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 65.4:
                        EFi_CO = 1.4082 * (V**-0.7728)
                    else:
                        EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0633 * (V**-1.0484)
                    else:
                        EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                    EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 1.2955 * (V**-0.7728)
                    else:
                        EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0570 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                    EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2012 <= car_birth <= 2013:
                    if V <= 65.4:
                        EFi_CO = 1.2550 * (V**-0.7728)
                    else:
                        EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0547 * (V**-1.0484)
                    else:
                        EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                    EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif car_birth == 2014:
                    if V <= 65.4:
                        EFi_CO = 1.2144 * (V**-0.7728)
                    else:
                        EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0524 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                    EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
                elif 2015 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 1.1739 * (V**-0.7728)
                    else:
                        EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                    if V <= 65.4:
                        EFi_HC = 0.0501 * (V**-1.0484)
                    else:
                        EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                    EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 11.249 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                    EFi_HC = 7.6244 * (V**-0.8364)
                    EFi_NOx = 2.6754 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 16.965 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                    EFi_HC = 3.0285 * (V**-0.7830)
                    EFi_NOx = 1.9923 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 65:
                        EFi_NH3 = 0.1
                    else:
                        EFi_NH3 = 0.07
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 1.8928 * (V**-0.7830)
                    EFi_NOx = 1.2352 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 9.9433 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                    EFi_HC = 0.9227 * (V**-0.7830)
                    EFi_NOx = 3.8859 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 45:
                        EFi_CO = 4.4952 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                    EFi_HC = 0.2958 * (V**-0.7830)
                    EFi_NOx = 0.4819 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 2000:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2001 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 11.920 * (V**-1.5356)
                    EFi_NOx = 3.0649 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 37.573 * (V**-1.2078)
                    EFi_HC = 3.1786 * (V**-1.5356)
                    EFi_NOx = 1.4931 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 32.899 * (V**-1.2078)
                    EFi_HC = 2.7387 * (V**-1.5356)
                    EFi_NOx = 1.1808 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.08
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '화물') and (car_size == '소형') ):
                if car_birth <= 1990:
                    EFi_CO = 36.169 * (V**-0.7587)
                    EFi_HC = 15.607 * (V**-1.0423)
                    EFi_NOx = 6.2007 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1991 <= car_birth <= 1996:
                    EFi_CO = 39.402 * (V**-0.8879)
                    EFi_HC = 23.400 * (V**-1.4041)
                    EFi_NOx = 7.5244 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 77.088 * (V**-1.2078)
                    EFi_HC = 18.731 * (V**-1.5356)
                    EFi_NOx = 4.4260 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 41.669 * (V**-1.2078)
                    EFi_HC = 14.190 * (V**-1.5356)
                    EFi_NOx = 3.4578 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V >= 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 36.578 * (V**-1.2078)
                    EFi_HC = 3.0337 * (V**-1.5356)
                    EFi_NOx = 2.0104 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2006 <= car_birth <= 2008:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V >= 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    EFi_CO = 14.202 * (V**-1.2078)
                    EFi_HC = 1.2233 * (V**-1.5356)
                    EFi_NOx = 0.2493 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V >= 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '중형'):
                if 1993 <= car_birth <= 1998:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif car_birth == 1999:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    EFi_NH3 = 0.002
                elif 2000 <= car_birth <= 2002: # CO, HC, NOx 배출계수 누락(⭕)
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.169
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.149
                    elif V > 90:
                        EFi_NH3 = 0.084
                elif 2003 <= car_birth <= 2008: # CO, HC, NOx 배출계수 누락(⭕)
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                    if V <= 60:
                        EFi_NH3 = 0.002
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.03
                    elif V > 90:
                        EFi_NH3 = 0.065
                elif 2009 <= car_birth:
                    if V <= 60:
                        EFi_NH3 = 0.004
                    elif 60 < V <= 90:
                        EFi_NH3 = 0.008
                    elif V > 90:
                        EFi_NH3 = 0.022
            elif (car_type == '화물') and (car_size == '대형'):
                if 1993 <= car_birth:
                    if V <= 65:
                        EFi_CO = 70
                        EFi_HC = 7
                        EFi_NOx = 4.5
                    else:
                        EFi_CO = 55
                        EFi_HC = 3.5
                        EFi_NOx = 7.5
                    if V < 85:
                        EFi_PM10 = 0.00030
                        EFi_PM2_5 = k * 0.00030
                    else:
                        EFi_PM10 = 0.00075
                        EFi_PM2_5 = k * 0.00075
                else:
                    EFi_CO = 1
                    EFi_HC = 1
                    EFi_NOx = 1
                    EFi_PM10 = 1
                    EFi_PM2_5 = 1
                EFi_NH3 = 0.002
        elif fuel == '경유':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)      # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)     # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416) # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.7392 * (V**-0.7524)
                    EFi_HC = 0.0989 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0839 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5775 * (V**-0.7524)
                    EFi_HC = 0.0825 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0420 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)        # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)   # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)           # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '중형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.6930 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.5414 * (V**-0.7524)
                    EFi_HC = 0.0927 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0396 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)            # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)      # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승용') and (car_size == '대형'):
                if car_birth <= 2004:
                    EFi_CO = 5.9672 * (V**-0.9534)
                    EFi_HC = 0.6523 * (V**-1.0167)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    if V <= 65:
                        EFi_PM10 = 0.3861 * (V**-0.5093)
                        EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                    else:
                        EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                        EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_NH3 = 0.001
                elif car_birth == 2005:
                    EFi_CO = 0.9609 * (V**-0.7524)
                    EFi_HC = 0.1865 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0723 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2006 <= car_birth <= 2010:
                    EFi_CO = 0.7507 * (V**-0.7524)
                    EFi_HC = 0.1554 * (V**-0.6848)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.0361 * (V**-0.3420)
                    EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
                    EFi_NH3 = 0.001
                elif 2011 <= car_birth <= 2015:
                    EFi_CO = 0.5141 * (V**-0.6792)
                    EFi_HC = 0.3713 * (V**-0.7513)
                    EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
                elif 2016 <= car_birth:
                    EFi_CO = 0.4574 * (V**-0.5215)
                    EFi_HC = 0.1300 * (V**-0.7265)
                    EFi_NOx = 2.7702 * (V**-0.3869)
                    if V <= 65.4:
                        EFi_PM10 = 0.0225 * (V**-0.7264)
                        EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                    else:
                        EFi_PM10 = 0.0009 * (V**0.0416)              # EFi_PM10 = 0.0009 * (V**0.0416) (⭕)
                        EFi_PM2_5 = k * 0.0009 * (V**0.0416)         # EFi_PM2_5 = k * 0.0009 * (V**0.0416) (⭕)
                    EFi_NH3 = 0.0019
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 0.9835 * (V**-0.5096)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 1.1412 * (V**-0.4324)
                    EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
                elif 1991 <= car_birth <= 1995:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.6313 * (V**-0.7298)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5999 * (V**-0.3294)
                    EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
                elif 1996 <= car_birth <= 1997:
                    if V <= 65.4:
                        EFi_CO = 3.4539 * (V**-0.4266)
                    else:
                        EFi_CO = 0.0051 * V + 0.2212
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6408 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.5168 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 3.7564 * (V**-0.5175)
                    EFi_HC = 1.1293 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2894 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)         # EFi_CO = 0.01166 * (V**0.09222) (⭕)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.371)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)         # EFi_CO = 0.01166 * (V**0.09222) (⭕)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.371)
                        EFi_HC = 0.829 * (V**-1.0961)
                    if V <= 65.4:
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_NOx = 0.0271 * (V**0.7596)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 32.550 * (V**-0.4944)
                    EFi_HC = 15.753 * (V**-0.5912)
                    if V < 80:
                        EFi_NOx = 40.692 * (V**-0.5590)
                    else:
                        EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                    EFi_PM10 = 5.4886 * (V**-0.5911)
                    EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 22.804 * (V**-0.4660)
                    else:
                        EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 16.410 * (V**-0.3790)
                    EFi_HC = 4.2324 * (V**-0.3926)
                    if V < 80:
                        EFi_NOx = 25.708 * (V**-0.4772)
                    else:
                        EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                    EFi_PM10 = 1.6593 * (V**-0.3935)
                    EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 16.378 * (V**-0.5340)
                    EFi_HC = 5.8477 * (V**-0.5466)
                    if V < 80:
                        EFi_NOx = 25.436 * (V**-0.4656)
                    else:
                        EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                    EFi_PM10 = 1.2848 * (V**-0.4715)
                    EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    EFi_NOx = 15.001 * (V**-0.4528)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                        EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                        EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif (car_type == '승합') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 28.205 * (V**-0.5337)
                    EFi_HC = 6.1146 * (V**-0.4979)
                    EFi_NOx = 41.346 * (V**-0.3645)
                    EFi_PM10 = 5.2158 * (V**-0.5048)
                    EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 2.4911 * (V**-0.4149)
                    EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 23.205 * (V**-0.5425)
                    EFi_HC = 6.5657 * (V**-0.5431)
                    EFi_NOx = 42.1379 * (V**-0.3786)
                    EFi_PM10 = 1.4432 * (V**-0.3870)
                    EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 0.9375 * (V**-0.3910)
                    EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 21.348 * (V**-0.5806)
                    EFi_HC = 6.6390 * (V**-0.5760)
                    EFi_NOx = 36.7191 * (V**-0.3548)
                    EFi_PM10 = 1.1507 * (V**-0.4804)
                    EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 9.6452 * (V**-0.5291)
                    EFi_HC = 3.2339 * (V**-0.7436)
                    EFi_NOx = 30.5870 * (V**-0.3548)
                    if V <= 80:
                        EFi_PM10 = 0.4657 * (V**-0.5634)
                        EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                    else:
                        EFi_PM10 = 0.0014 * (V**0.7970)
                        EFi_PM2_5 = k * 0.0014 * (V**0.7970)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.003
                elif 2015 <= car_birth:
                    EFi_CO = 11.4415 * (V**-0.8036)
                    EFi_HC = 0.6774 * (V**-0.8321)
                    EFi_NOx = 112.1229 * (V**-1.6393)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 4.5854 * (V**-0.3613)
                    EFi_HC = 0.4840 * (V**-0.2756)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.8117 * (V**-0.4071)
                    EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
                elif 1991 <= car_birth <= 1995:
                    EFi_CO = 3.4774 * (V**-0.3483)
                    EFi_HC = 0.4844 * (V**-0.3288)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6188 * (V**-0.4540)
                    EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 3.3934 * (V**-0.3837)
                    EFi_HC = 0.4955 * (V**-0.3393)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.7037 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
                elif 1998 <= car_birth <= 1999:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.6157 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
                elif 2000 <= car_birth <= 2003:
                    EFi_CO = 4.0896 * (V**-0.6083)
                    EFi_HC = 0.6122 * (V**-0.5684)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.4838 * (V**-0.5357)
                    EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
                elif 2004 <= car_birth <= 2007:
                    EFi_CO = 3.2797 * (V**-0.8887)
                    EFi_HC = 0.1807 * (V**-0.6588)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.2067 * (V**-0.3596)
                    EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
                elif 2008 <= car_birth <= 2011:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.374)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_NOx = 24.3491 * (V**-0.7277)
                    EFi_PM10 = 0.3111 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
                elif 2012 <= car_birth <= 2016:
                    if V <= 65.4:
                        EFi_CO = 4.222 * (V**-1.4035)
                        EFi_NOx = 2.0217 * (V**-0.2645)
                    else:
                        EFi_CO = 0.01166 * (V**0.09222)
                        EFi_NOx = 0.0271 * (V**0.7596)
                    if V <= 97.3:                               # 97.3 초과에 대한 내용 없음.(편람 p.374)
                        EFi_HC = 0.829 * (V**-1.0961)
                    EFi_PM10 = 0.1119 * (V**-0.5125)
                    EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
                EFi_NH3 = 0.001
            elif (car_type == '화물') and (car_size == '중형'):
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100) (⭕)     
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)        # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                elif 2008 <= car_birth <= 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                EFi_NH3 = 0.001
            elif car_type == '특수':
                if car_birth <= 1995:
                    EFi_CO = 16.769 * (V**-0.3772)
                    EFi_HC = 6.7755 * (V**-0.5003)
                    EFi_NOx = 24.915 * (V**-0.3942)
                    EFi_PM10 = 3.6772 * (V**-0.5514)
                    EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 2000:
                    EFi_CO = 21.057 * (V**-0.4958)
                    EFi_HC = 6.7532 * (V**-0.5711)
                    EFi_NOx = 25.022 * (V**-0.4240)
                    EFi_PM10 = 3.5285 * (V**-0.5962)
                    EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
                    EFi_NH3 = 0.003
                elif car_birth == 2001:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.4444 * (V**-0.4824)
                    EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
                    EFi_NH3 = 0.003
                elif 2002 <= car_birth <= 2004:
                    EFi_CO = 23.501 * (V**-0.6100)             # EFi_CO = 23.501 * (V**-0.6100) (⭕)
                    EFi_HC = 6.8738 * (V**-0.5913)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 1.0432 * (V**-0.4992)
                    EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 15.256 * (V**-0.7448)
                    EFi_HC = 2.0502 * (V**-0.6504)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.2979 * (V**-0.4008)
                    EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
                    EFi_NH3 = 0.003
                elif 2008 <= car_birth <= 2009:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.003
                elif car_birth == 2010:
                    EFi_CO = 8.1771 * (V**-0.7725)
                    EFi_HC = 1.2991 * (V**-0.6538)
                    if V <= 64.7:
                        EFi_NOx = 17.3032 * (V**-0.3660)
                        EFi_PM10 = 0.0539 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                    else:
                        EFi_NOx = 0.3259 * (V**0.5773)
                        EFi_PM10 = 0.0002 * V - 0.00756
                        EFi_PM2_5 = k * 0.0002 * V - 0.00756
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 4.5201 * (V**-0.7279)
                    EFi_HC = 1.6826 * (V**-0.8045)
                    if V <= 64.7:
                        EFi_NOx = 17.2485 * (V**-0.4040)
                        EFi_PM10 = 0.0469 * (V**-0.4674)
                        EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                    else:
                        EFi_NOx = 1.1797 * (V**0.2308)
                        EFi_PM10 = 0.000168 * V - 0.00516
                        EFi_PM2_5 = k * 0.000168 * V - 0.00516
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 7.4065 * (V**-0.5995)
                    EFi_HC = 2.4562 * (V**-1.3145)
                    EFi_NOx = 42.7393 * (V**-1.2949)
                    if V <= 64.7:
                        EFi_PM10 = 0.0081 * (V**-0.5182)
                        EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                    else:
                        EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                        EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
                    EFi_NH3 = 0.007
            elif (car_type == '화물') and (car_size == '대형'):
                if car_birth <= 1995:
                    EFi_CO = 30.402 * (V**-0.4685)
                    EFi_HC = 15.75 * (V**-0.582)
                    EFi_NOx = 117.49 * (V**-0.365)
                    EFi_PM10 = 7.6212 * (V**-0.4183)
                    EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
                    EFi_NH3 = 0.003
                elif 1996 <= car_birth <= 1997:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 6.0264 * (V**-0.4627)
                    EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
                    EFi_NH3 = 0.003
                elif 1998 <= car_birth <= 2000:
                    EFi_CO = 18.101 * (V**-0.3454)
                    EFi_HC = 10.301 * (V**-0.5856)
                    EFi_NOx = 94.319 * (V**-0.4061)
                    EFi_PM10 = 4.873 * (V**-0.4382)
                    EFi_PM2_5 = k * 4.873 * (V**-0.4382)
                    EFi_NH3 = 0.003
                elif 2001 <= car_birth <= 2004:
                    EFi_CO = 28.399 * (V**-0.5999)
                    EFi_HC = 10.031 * (V**-0.5828)
                    EFi_NOx = 85.301 * (V**-0.4023)
                    EFi_PM10 = 3.7541 * (V**-0.4055)
                    EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
                    EFi_NH3 = 0.003
                elif 2005 <= car_birth <= 2007:
                    EFi_CO = 52.136 * (V**-0.8618)
                    EFi_HC = 3.7878 * (V**-0.5425)
                    EFi_NOx = 107.5 * (V**-0.5679)
                    EFi_PM10 = 2.6847 * (V**-0.6112)
                    EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
                    EFi_NH3 = 0.003
                elif car_birth == 2008:                    # 2008 <= car_birth <= 2009 -> 2008 == car_birth: 변경 (⭕)
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.003
                elif 2009 <= car_birth <= 2010:
                    EFi_CO = 6.8493 * (V**-0.6506)
                    EFi_HC = 1.7177 * (V**-0.6781)
                    EFi_NOx = 40.7564 * (V**-0.4757)
                    EFi_PM10 = 0.2418 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
                    EFi_NH3 = 0.011
                elif 2011 <= car_birth <= 2014:
                    EFi_CO = 5.4607 * (V**-0.2990)
                    EFi_HC = 0.8863 * (V**-0.6933)
                    EFi_NOx = 40.3729 * (V**-0.5386)
                    EFi_PM10 = 0.2125 * (V**-0.4650)
                    EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
                    EFi_NH3 = 0.011
                elif 2015 <= car_birth:
                    EFi_CO = 4.3762 * (V**-0.4550)
                    EFi_HC = 0.3627 * (V**-0.7071)
                    EFi_NOx = 18.0405 * (V**-1.0986)
                    EFi_PM10 = 0.0363 * (V**-0.4727)
                    EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
                    EFi_NH3 = 0.007
        elif fuel == 'LPG':
            if (car_type == '승용') and (car_size == '경형'):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
            elif (car_type == '승용') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 101.79 * (V**-1.6823)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 7.5371 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 11.173 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    EFi_CO = 44.956 * (V**-1.0085)
                    EFi_HC = 3.2821 * (V**-1.3927)
                    EFi_NOx = 4.7108 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 39.362 * (V**-1.0085)
                    EFi_HC = 2.8981 * (V**-1.3927)
                    EFi_NOx = 1.8419 * (V**-0.7864)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최이사님 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth <= 2007:
                    EFi_CO = 73.022 * (V**-1.2078)
                    EFi_HC = 4.4166 * (V**-1.5356)
                    EFi_NOx = 2.0280 * (V**-0.7978)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2008:
                    if V <= 79.6:
                        EFi_CO = 0.7693 * (V**-0.7666)
                        EFi_HC = 0.1063 * (V**-1.0745)
                    else:
                        EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                        EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                    EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2009 <= car_birth <= 2011:
                    if V <= 79.6:
                        EFi_CO = 0.7059 * (V**-0.7666)
                        EFi_HC = 0.0974 * (V**-1.0745)
                    else:
                        EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                        EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2012 <= car_birth <= 2013:
                    if V <= 79.6:
                        EFi_CO = 0.6830 * (V**-0.7666)
                        EFi_HC = 0.0943 * (V**-1.0745)
                    else:
                        EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif car_birth == 2014:
                    if V <= 79.6:
                        EFi_CO = 0.6602 * (V**-0.7666)
                        EFi_HC = 0.0911 * (V**-1.0745)
                    else:
                        EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2015 <= car_birth:
                    if V <= 79.6:
                        EFi_CO = 0.6374 * (V**-0.7666)
                        EFi_HC = 0.0879 * (V**-1.0745)
                    else:
                        EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                        EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                    EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
                if car_birth <= 1996:
                    if V <= 45:
                        EFi_CO = 22.498 * (V**-0.6579)
                    else:
                        EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                    EFi_HC = 12.961 * (V**-0.8364)
                    EFi_NOx = 4.0131 * (V**-0.3236)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth == 1999:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 1.8528 * (V**-0.3889)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2002:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 2.2714 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2003 <= car_birth <= 2005:
                    if V <= 45:
                        EFi_CO = 19.887 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                    EFi_HC = 1.1073 * (V**-0.7830)
                    EFi_NOx = 5.8289 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 45:
                        EFi_CO = 8.9904 * (V**-0.8461)
                    else:
                        EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                    EFi_HC = 0.3549 * (V**-0.7830)
                    EFi_NOx = 0.7228 * (V**-0.9198)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            elif (car_type == '승합') and (car_size == '소형'):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 6.1701 * (V**-1.0719)
                        EFi_HC = 2.3782 * (V**-1.9979)
                    else:
                        EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                        EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                    if V <= 97.3:
                        EFi_NOx = 0.4758 * (V**-1.0665)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
            # 최이사님 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
            elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
                if car_birth <= 1990:
                    EFi_CO = 72.338 * (V**-0.7587)
                    EFi_HC = 26.532 * (V**-1.0423)
                    EFi_NOx = 9.3011 * (V**-0.6781)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1991 <= car_birth == 1996:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 26.520 * (V**-1.4041)
                    EFi_NOx = 11.287 * (V**-0.7634)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 1997 <= car_birth <= 1999:
                    EFi_CO = 29.825 * (V**-0.6771)
                    EFi_HC = 4.7595 * (V**-0.9512)
                    EFi_NOx = 12.562 * (V**-0.8606)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2000 <= car_birth <= 2005:
                    EFi_CO = 17.829 * (V**-0.6778)
                    EFi_HC = 6.3668 * (V**-1.2849)
                    EFi_NOx = 8.8952 * (V**-0.8119)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                elif 2006 <= car_birth:
                    if V <= 65.4:
                        EFi_CO = 16.491 * (V**-1.4713)
                        EFi_HC = 1.9218 * (V**-1.8418)
                    else:
                        EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                        EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                    if V <= 97.3:
                        EFi_NOx = 0.1246 * (V**-0.441)
                    if V < 85:
                        EFi_PM10 = 0.0002
                        EFi_PM2_5 = k * 0.0002
                    else:
                        EFi_PM10 = 0.0005
                        EFi_PM2_5 = k * 0.0005
                if V <= 60:
                    EFi_NH3 = 0.0095
                elif 60 < V <= 90:
                    EFi_NH3 = 0.0082
                else:
                    EFi_NH3 = 0.022
        # else:
        #     # 값 고민 : 1
        #     # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        #     EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

        if V < 60:
            EFi_COm += EFi_CO*0.34
            EFi_HCm += EFi_HC*0.34
            EFi_NOxm += EFi_NOx*0.34
            EFi_PM10m += EFi_PM10*0.34
            EFi_PM2_5m += EFi_PM2_5*0.34
            EFi_NH3m += EFi_NH3*0.34
        elif 60 <= V < 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33
        elif V > 90:
            EFi_COm += EFi_CO*0.33
            EFi_HCm += EFi_HC*0.33
            EFi_NOxm += EFi_NOx*0.33
            EFi_PM10m += EFi_PM10*0.33
            EFi_PM2_5m += EFi_PM2_5*0.33
            EFi_NH3m += EFi_NH3*0.33

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_COm)
    EFi_HC_list.append(EFi_HCm)
    EFi_NOx_list.append(EFi_NOxm)
    EFi_PM10_list.append(EFi_PM10m)
    EFi_PM2_5_list.append(EFi_PM2_5m)
    EFi_NH3_list.append(EFi_NH3m)

100%|██████████| 25117098/25117098 [02:40<00:00, 156255.75it/s]


In [36]:
df2.shape[0]

25117098

In [37]:
len(EFi_CO_list), len(EFi_HC_list), len(EFi_NOx_list), len(EFi_PM10_list), len(EFi_PM2_5_list), len(EFi_NH3_list)

(25117098, 25117098, 25117098, 25117098, 25117098, 25117098)

In [38]:
# 28.1s
df2['EFi_CO'] = EFi_CO_list
df2['EFi_HC'] = EFi_HC_list
df2['EFi_NOx'] = EFi_NOx_list
df2['EFi_PM10'] = EFi_PM10_list
df2['EFi_PM2_5'] = EFi_PM2_5_list
df2['EFi_NH3'] = EFi_NH3_list

In [39]:
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

#### 등급별 확인

In [40]:
grp =  df2.groupby('등급').agg({'EFi_CO':'sum', 'EFi_HC':'sum', 'EFi_NOx':'sum', 'EFi_PM10':'sum', 'EFi_PM2_5':'sum', '차대번호':'count'})
grp = grp.rename(columns={'차대번호':'차량대수'})
grp['EFi_CO_per'] = grp['EFi_CO'] / grp['차량대수']
grp['EFi_HC_per'] = grp['EFi_HC'] / grp['차량대수']
grp['EFi_NOx_per'] = grp['EFi_NOx'] / grp['차량대수']
grp['EFi_PM10_per'] = grp['EFi_PM10'] / grp['차량대수']
grp['EFi_PM2_5_per'] = grp['EFi_PM2_5'] / grp['차량대수']
grp

,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,차량대수,EFi_CO_per,EFi_HC_per,EFi_NOx_per,EFi_PM10_per,EFi_PM2_5_per
등급,,,,,,,,,,,
1,41369.40,703.16,12974.11,219.53,201.97,2376172,0.02,0.00,0.01,0.00,0.00
2,495465.47,9443.24,58008.10,1096.68,1009.15,11182913,0.04,0.00,0.01,0.00,0.00
3,415061.17,30356.37,1597387.10,7077.88,6471.65,8753079,0.05,0.00,0.18,0.00,0.00
4,159972.19,18498.24,620910.08,6826.87,6251.64,1511032,0.11,0.01,0.41,0.00,0.00
5,421637.05,92091.89,1103119.97,48655.36,46254.81,1154641,0.37,0.08,0.96,0.04,0.04
X,6520.31,387.85,6032.44,104.17,98.83,139261,0.05,0.00,0.04,0.00,0.00


### 배출량 계산

In [41]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00


In [42]:
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df2['E_HOT_CO'] = df2['VKT'] * (df2['EFi_CO'] / 1000) * df2['DF_CO'] * (1 - df2['R_CO'] / 100)
df2['E_HOT_HC'] = df2['VKT'] * (df2['EFi_HC'] / 1000) * df2['DF_HC'] * (1 - df2['R_HC'] / 100)
df2['E_HOT_NOx'] = df2['VKT'] * (df2['EFi_NOx'] / 1000) * df2['DF_NOx'] * (1 - 0 / 100)
df2['E_HOT_PM10'] = df2['VKT'] * (df2['EFi_PM10'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
df2['E_HOT_PM2_5'] = df2['VKT'] * (df2['EFi_PM2_5'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)

In [43]:
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [44]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col + check_E_HOT_col].head()

,fuel,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,...,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5
0,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.20,0.00,0.06,0.00,0.00
1,휘발유,승용,중형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.20,0.00,0.06,0.00,0.00
2,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.36,0.00,0.08,0.00,0.00
3,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.36,0.00,0.08,0.00,0.00
4,휘발유,승용,대형,2023,0,1.00,1.00,1.00,1.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.36,0.00,0.08,0.00,0.00


In [45]:
# 소수점 옵션
pd.options.display.float_format = '{:.5f}'.format
# pd.reset_option('display.float_format')

In [46]:
# 속력 : 75
grp =  df2.groupby('등급').agg({'E_HOT_CO':'sum', 'E_HOT_HC':'sum', 'E_HOT_NOx':'sum', 'E_HOT_PM10':'sum', 'E_HOT_PM2_5':'sum', '차대번호':'count'})
grp = grp.rename(columns={'차대번호':'차량대수'})
grp['E_HOT_CO_per'] = grp['E_HOT_CO'] / grp['차량대수']
grp['E_HOT_HC_per'] = grp['E_HOT_HC'] / grp['차량대수']
grp['E_HOT_NOx_per'] = grp['E_HOT_NOx'] / grp['차량대수']
grp['E_HOT_PM10_per'] = grp['E_HOT_PM10'] / grp['차량대수']
grp['E_HOT_PM2_5_per'] = grp['E_HOT_PM2_5'] / grp['차량대수']
grp

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
등급,,,,,,,,,,,
1,571222.56963,9700.68266,178902.01834,3024.75588,2782.77541,2376172,0.24040,0.00408,0.07529,0.00127,0.00117
2,7866246.53597,162646.62056,935286.18664,16967.40713,15619.47391,11182913,0.70342,0.01454,0.08364,0.00152,0.00140
3,3545190.00487,169377.57918,27127273.77246,25642.11166,23427.16330,8753079,0.40502,0.01935,3.09917,0.00293,0.00268
4,4393063.62347,496522.26701,12637755.17992,163901.96495,150002.99155,1511032,2.90733,0.32860,8.36366,0.10847,0.09927
5,10294630.40486,2231577.62522,23632252.06653,1122256.38376,1062374.24083,1154641,8.91587,1.93270,20.46719,0.97195,0.92009
X,128542.75443,9486.11621,105620.01537,2504.19955,2365.65437,139261,0.92303,0.06812,0.75843,0.01798,0.01699


## 자동차-엔진 미가열(Cold-start) 배출

### 배출량 식
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

- $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
- $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
- $N_j$ : 차종 $j$의 수
- $M_j$ : 차종 $j$의 주행거리
- $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
- $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

### 베타($\beta$)
- 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
    - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
    - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

    - 1회 주행거리 : 12.35 km (2002년 기준)
    - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
    - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

- 산정식

-|Factor $\beta$의 산정식
-|-
$Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
$Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

- $T_a$ : 대기온도
- $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)

In [47]:
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온

In [48]:
df2['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df2['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [49]:
df2['Beta(Estimated)'].head()

0   0.27643
1   0.27643
2   0.27643
3   0.27643
4   0.27643
Name: Beta(Estimated), dtype: float64

In [50]:
df2['Beta(Estimated)'].isnull().sum()

0

### $e^{COLD} / e^{HOT}$

In [51]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [52]:
df2['차량연식'].describe()

count   25117098.00000
mean        2014.35604
std            6.18721
min         1900.00000
25%         2011.00000
50%         2015.00000
75%         2019.00000
max         2023.00000
Name: 차량연식, dtype: float64

In [53]:
# 1m 31.7s
# 휘발유(자동 제어)
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df2.loc[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df2.loc[df2['fuel'] == '휘발유', 'eCOLD/eHOT_PM'] = 1
# 경유
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df2.loc[(df2['fuel'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta
# LPG
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df2.loc[(df2['fuel'] == 'LPG') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1

In [54]:
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     4282101
eCOLD/eHOT_NOx    4282101
eCOLD/eHOT_HC     4282101
eCOLD/eHOT_PM     4248868
dtype: int64

In [55]:
df2[check_ecoldhot_col] = df2[check_ecoldhot_col].fillna(1)
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     0
eCOLD/eHOT_NOx    0
eCOLD/eHOT_HC     0
eCOLD/eHOT_PM     0
dtype: int64

In [56]:
df2.loc[df2['eCOLD/eHOT_CO'] < 0, 'eCOLD/eHOT_CO'].shape

(0,)

In [57]:
df2.loc[df2['eCOLD/eHOT_NOx'] < 0, 'eCOLD/eHOT_NOx'].shape

(0,)

In [58]:
df2.loc[df2['eCOLD/eHOT_HC'] < 0, 'eCOLD/eHOT_HC'].shape

(0,)

In [59]:
df2.loc[df2['eCOLD/eHOT_PM'] < 0, 'eCOLD/eHOT_PM'].shape

(0,)

### 배출량 계산

In [60]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM'],
      dtype='object')

#### 🔺 계산식 고민
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
- $M_j$ : 차종 $j$의 주행거리
    - 1년 주행거리로 계산? 
        - VKT
    - 1회 주행거리로 계산?
        - l_trip

In [61]:
#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df2['E_COLD_CO'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_CO'] * (df2['eCOLD/eHOT_CO'] - 1)
df2['E_COLD_NOx'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_NOx'] * (df2['eCOLD/eHOT_NOx'] - 1)
df2['E_COLD_HC'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_HC'] * (df2['eCOLD/eHOT_HC'] - 1)
df2['E_COLD_PM10'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM10'] * (df2['eCOLD/eHOT_PM'] - 1)
df2['E_COLD_PM2_5'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM2_5'] * (df2['eCOLD/eHOT_PM'] - 1)

In [62]:
df2.loc[df2['E_COLD_NOx'] < 0, ['fuel', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].shape

(1758697, 6)

In [63]:
df2.loc[df2['E_COLD_NOx'] < 0, ['fuel', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].head()

,fuel,Beta(Estimated),VKT,EFi_NOx,eCOLD/eHOT_NOx,E_COLD_NOx
5,LPG,0.27643,12811.50000,0.00793,0.90560,-2.65127
6,LPG,0.27643,13578.00000,0.00793,0.90560,-2.80989
7,LPG,0.27643,12775.00000,0.00793,0.90560,-2.64371
8,LPG,0.27643,13906.50000,0.00793,0.90560,-2.87787
9,LPG,0.27643,14855.50000,0.00793,0.90560,-3.07426


In [64]:
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']
df2[check_E_HOT_col + check_E_cold_col].head()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
0,0.19992,0.00349,0.06213,0.00128,0.00117,382.65079,44.41041,10.45843,0.00000,0.00000
1,0.20445,0.00357,0.06354,0.00130,0.00120,391.32276,45.41687,10.69545,0.00000,0.00000
2,0.36373,0.00459,0.08167,0.00168,0.00154,696.18895,58.37515,13.74706,0.00000,0.00000
3,0.36373,0.00459,0.08167,0.00168,0.00154,696.18895,58.37515,13.74706,0.00000,0.00000
4,0.35746,0.00451,0.08026,0.00165,0.00152,684.18569,57.36868,13.51004,0.00000,0.00000


In [65]:
df2[check_E_HOT_col + check_E_cold_col].tail()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
25117093,33.18436,10.64487,25.22250,3.71042,3.41359,0.00000,0.00000,0.00000,0.00000,0.00000
25117094,71.69223,17.17018,79.27736,7.50008,6.90008,0.00000,0.00000,0.00000,0.00000,0.00000
25117095,37.45177,11.97932,179.73337,11.62030,10.69068,0.00000,0.00000,0.00000,0.00000,0.00000
25117096,30.26327,7.24801,33.46517,3.16599,2.91271,0.00000,0.00000,0.00000,0.00000,0.00000
25117097,40.00359,12.79554,191.97973,12.41207,11.41910,0.00000,0.00000,0.00000,0.00000,0.00000


In [66]:
df2[check_E_cold_col].isnull().sum()

E_COLD_CO       0
E_COLD_NOx      0
E_COLD_HC       0
E_COLD_PM10     0
E_COLD_PM2_5    0
dtype: int64

## 자동차-휘발유 증발 배출

### 배출량 식
- 3가지
    - 주간증발손실(Diurnal loss)
    - 고온증발손실(Hot and warm soak)
    - 주행손실(Running loss)
- 현재('19 배출량 기준) 휘발유 증발 배출원
    - 주간증발손실, 주행손실 고려
    - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

---

$$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

$S^{fi} = q \times e^{fi} \times x$
$$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

- $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
- $m_j$ : 차종 j의 연간 총 주행거리
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

### 배출계수

#### 🔺 수정 중

#### $e^d$ Diurnal(g/day)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)

##### 🔺 RVP(증기압)
- 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
    - summer : 60
    - winter : 70
    - 평균 : 65
- 수정 사항(2023.04.20, 최)
    - 국내 과학기술원, 환경부 자료 찾아서 사용 추천

In [67]:
RVP = 65

##### ❗ t_a(기온)
- t_a, t_min, t_rise
    - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
        - t_a = (t_max + t_min) / 2
        - t_rise = t_max - t_min
    - 기상자료개방포털(2002년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)
    - 기상자료개방포털(2022년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 8°C(최저), 12.9°C(평균), 18.6°C(최고)

In [68]:
t_min = 8 # 평균 최저 온도
t_max = 18.6
t_a = (t_min + t_max) / 2
t_rise = 10.6 # 평균 상승 온도

In [69]:
df2.loc[df2['차량연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df2.loc[df2['차량연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [70]:
df2['e_d'].isnull().sum()

0

#### $e^{R,HOT}$ Hot running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [71]:
df2.loc[df2['차량연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [72]:
df2['e_RHOT'].isnull().sum()

0

#### $e^{S,HOT}$ Hot soak(g/procedure)
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수

In [73]:
df2.loc[df2['차량연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df2.loc[df2['차량연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{S,WARM}$ Waram soak(g/procedure)
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [74]:
df2.loc[df2['차량연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

#### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수

In [75]:
df2.loc[df2['차량연식'] < 1991, 'e_fi'] = 0.7
df2.loc[df2['차량연식'] >= 1991, 'e_fi'] = 0

#### $e^{R,WARM}$ Warm running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [76]:
df2.loc[df2['차량연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['차량연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [77]:
df2['e_RWARM'].isnull().sum()

0

### $R$
$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
- Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $m_j$ : 차종 j의 연간 총 주행거리
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $w$ : 미가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수

#### ❓ 확인 중
- 수정 사항(2023.04.20, 최이사님)
    - 찾는 중
- 수정 사항(2023.04.20, 최이사님)
    - w:p = 0.8:0.2

In [78]:
p = 0.2 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.8 # 미가열 엔진 상태로 끝나는 trip의 비율

In [79]:
df2['R'] = df2['VKT'] * (p * df2['e_RHOT'] + w * df2['e_RWARM'])

In [80]:
df2['R'].isnull().sum()

0

### $S^{fi}$
$S^{fi} = q \times e^{fi} \times x$
- Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [81]:
# 8.9s
not_injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] < 1991)].shape[0]
injection = df2[(df2['fuel'] == '휘발유') & (df2['차량연식'] >= 1991)].shape[0]
# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)
q

0.9931614094087922

In [82]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_fi'] = q * df2['e_fi'] * ( df2['VKT'] / (365 * l_trip) )

### $S^c$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
- Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [83]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_c'] = (1-q) * ( p * (df2['VKT'] / (365 * l_trip)) * df2['e_SHOT'] + w * (df2['VKT'] / (365 * l_trip)) * df2['e_SWARM'] )

### $E_{EVA,VOC}$
$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)

In [84]:
df2['E_EVA_VOC'] = 365 * (df2['e_d'] + df2['S_c'] + df2['S_fi']) + df2['R']

In [85]:
df2['E_EVA_VOC'].isnull().sum()

0

In [86]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [87]:
# 휘발유만 증발 배출 존재
df2.loc[(df2['fuel'] == 'LPG') | (df2['fuel'] == '경유'), 'E_EVA_VOC'] = 0

## 배출량 합계
$E_{total}(kg) = E_{HOT}(kg) + E_{COLD}(g) + E_{EVAP}(g)$

### issue
- LPG : E_COLD_NOx 값 음수 -> 0으로 처리

In [88]:
df2[df2['E_COLD_NOx'] < 0].shape

(1758697, 58)

In [89]:
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df2.loc[df2['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [90]:
df2[df2['E_COLD_NOx'] < 0].shape

(0, 58)

In [91]:
check_E_HOT_col, check_E_cold_col

(['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5'],
 ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5'])

In [92]:
df2.loc[(df2['fuel'] == '휘발유') & (df2['E_COLD_PM10'] != 0)].shape

(0, 58)

In [93]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [94]:
# 단위 변환(g -> kg)
df2['E_COLD_CO'] = df2['E_COLD_CO'] / 1000
df2['E_COLD_HC'] = df2['E_COLD_HC'] / 1000
df2['E_COLD_NOx'] = df2['E_COLD_NOx'] / 1000
df2['E_COLD_PM10'] = df2['E_COLD_PM10'] / 1000
df2['E_COLD_PM2_5'] = df2['E_COLD_PM2_5'] / 1000
df2['E_EVA_VOC'] = df2['E_EVA_VOC'] / 1000

In [95]:
# 26.5s
fuel = '휘발유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC'] + df2.loc[df2['fuel'] == fuel , 'E_EVA_VOC']
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']

In [96]:
# 22.7s
fuel = '경유'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']

In [97]:
# 20.0s
fuel = 'LPG'
df2.loc[df2['fuel'] == fuel, 'E_CO_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_CO'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_CO']
df2.loc[df2['fuel'] == fuel, 'E_HC_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_HC'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_HC']
df2.loc[df2['fuel'] == fuel, 'E_NOx_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_NOx']
df2.loc[df2['fuel'] == fuel, 'E_PM10_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM10']
df2.loc[df2['fuel'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['fuel'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['fuel'] == fuel , 'E_COLD_PM2_5']

In [98]:
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']
df2[check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,0.58257,0.36277,0.10655,0.00128,0.00117
1,0.59577,0.36449,0.10896,0.00130,0.00120
2,1.05992,0.38665,0.14005,0.00168,0.00154
3,1.05992,0.38665,0.14005,0.00168,0.00154
4,1.04165,0.38492,0.13763,0.00165,0.00152


## 황산화물 배출량

In [99]:
# 46.2s
fcc_list = []
for fone, cd, ty in tqdm(df2[['fuel', '차종', '차종유형']].values):
    fccm = 0
    for V in [35, 70, 100]:
        if fone == '휘발유':
            if cd == '승용':
                fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
            elif cd == '승합':
                if ty =='경형':
                    fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
                elif ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
            elif cd == '화물':
                if ty == '경형':   
                    fcc = 0.01090 * (V**2) - 1.5100*V + 93.672 # E01
                elif ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
            elif cd == '특수':
                if ty == '소형' or ty == '중형' or ty == '대형':
                    fcc = 0.01870 * (V**2) - 2.6974*V + 156.77 # E04
        elif fone == '경유':
            if cd == '승용':
                fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
            elif cd == '승합':
                if ty == '경형' or ty == '소형':
                    fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
                elif ty == '중형':
                    if V <= 60:
                        fcc = 1425.2 * (V**(-0.7593)) # E17
                    else:
                        fcc = 0.0082 * (V**2) - 0.0430*V + 60.12 # E17
                elif ty == '대형':
                    if V <= 60:
                        fcc = 1919.0 * (V**(-0.5396)) # E16
                    else:
                        fcc = 0.0447 * (V**2) - 7.072*V + 478 # E16
            elif cd == '화물':
                if ty == '경형' or ty == '소형':
                    fcc = 0.00790 * (V**2) - 1.3123*V + 83.660 # E12
                elif ty == '중형':
                    if V <= 60:
                        fcc = 1068.4 * (V**(-0.4905)) # E18
                    else:
                        fcc = 0.0126 * (V**2) - 0.6589*V + 141.2 # E18
                elif ty == '대형':
                    if V <= 60:
                        fcc = 1595.1 * (V**(-0.4744)) # E19
                    else:
                        fcc = 0.0382 * (V**2) - 5.1630*V + 399.3 # E19
            elif cd == '특수':
                if ty == '소형':
                    if V <= 60:
                        fcc = 1068.4 * (V**(-0.4905)) # E18
                    else:
                        fcc = 0.0126 * (V**2) - 0.6589*V + 141.2 # E18
                if ty == '중형' or ty == '대형':
                    if V <= 60:
                        fcc = 1595.1 * (V**(-0.4744)) # E19
                    else:
                        fcc = 0.0382 * (V**2) - 5.1630*V + 399.3 # E19
        elif fone == 'LPG':
            if cd == '승용':
                fcc = 45 # E09
            elif cd == '승합':
                fcc = 45 # E09
            elif cd == '화물':
                if ty == '경형' or ty == '소형' or ty == '중형':
                    fcc = 45 # E09
            elif cd == '특수':
                if ty == '소형' or ty == '중형'or ty == '대형':
                    fcc = 45 # E09
        if V < 60:
            fccm += fcc * 0.34
        elif 60 <= V < 90:
            fccm += fcc * 0.33
        elif V > 90:
            fccm += fcc * 0.33
    fcc_list.append(fccm)

100%|██████████| 25117098/25117098 [01:07<00:00, 370403.86it/s]


In [100]:
df2.shape

(25117098, 63)

In [101]:
len(fcc_list)

25117098

In [102]:
# 5.0s
# 단위 : %
sf_list = []
for fone in tqdm(df2['fuel']):
    if fone == '경유':
        sf = 0.017
    elif fone == '휘발유':
        sf = 0.001
    elif fone == 'LPG':
        sf = 0.015
    sf_list.append(sf)

  2%|▏         | 499563/25117098 [00:00<00:14, 1697064.29it/s]

100%|██████████| 25117098/25117098 [00:13<00:00, 1847452.00it/s]


In [103]:
df2.shape, len(sf_list)

((25117098, 63), 25117098)

In [104]:
# 3.5s
df2['fcc'] = fcc_list
df2['sf'] = sf_list

In [105]:
df2['E_SOx_total'] = df2['VKT'] * df2['fcc'] * df2['sf'] * 2 / (1000*100)

## NH3

In [106]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total'],
      dtype='object')

In [107]:
df2['VKT'].describe()

count   25117098.00000
mean       14437.04352
std         3300.01418
min         2847.00000
25%        13030.50000
50%        13505.00000
75%        14271.50000
max        98185.00000
Name: VKT, dtype: float64

In [108]:
df2['EFi_NH3'].isnull().sum()

0

In [109]:
df2['EFi_NH3'].describe()

count   25117098.00000
mean           0.00278
std            0.00491
min            0.00033
25%            0.00063
50%            0.00264
75%            0.00264
max            0.04917
Name: EFi_NH3, dtype: float64

In [110]:
df2['E_NH3_total'] = df2['VKT'] * df2['EFi_NH3'] / 1000

In [111]:
df2['E_NH3_total'].describe()

count   25117098.00000
mean           0.03823
std            0.06593
min            0.00223
25%            0.00828
50%            0.03421
75%            0.03642
max            1.73189
Name: E_NH3_total, dtype: float64

## CO2

### 계산식

In [112]:
# 
fc_list = []
for ty, yr, fuel, cc, twg in tqdm(df2[['차종유형', '차량연식', 'fuel', '배기량', '총중량']].values):
    fcm = 0
    for V in [35, 70, 100]:
        if fuel == '휘발유' and ty != '경형' and ty !='대형':
            if yr <= 1971:
                if cc < 1400:
                    if 10 <= V < 60:
                        fc = 521 * V**(-0.554)
                    elif 60 <= V < 80:
                        fc = 55
                    elif 80 <= V <130:
                        fc = 0.386*V + 24143
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 60:
                        fc = 681 * V**(-0.583)
                    elif 60 <= V < 80:
                        fc = 67
                    elif 80 <= V <130:
                        fc = 0.471*V + 29286
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 60:
                        fc = 979 * V**(-0.628)
                    elif 60 <= V < 80:
                        fc = 80
                    elif 80 <= V <130:
                        fc = 0.414*V + 46867
                    else:
                        fc = 0
                else:
                    fc = 0
            elif 1972 <= yr <= 1977:
                if cc < 1400:
                    if 10 <= V < 60:
                        fc = 595 * V**(-0.63)
                    elif 60 <= V < 130:
                        fc = 95 - 1.324*V + 0.0086*V**2
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 60:
                        fc = 864 * V**(-0.69)
                    elif 60 <= V < 130:
                        fc = 59 - 0.407*V + 0.0042*V**2
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 60:
                        fc = 1236 * V**(-0.764)
                    elif 60 <= V < 130:
                        fc = 65 - 0.407*V + 0.0042*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            elif 1978 <= yr <= 1985:
                if cc < 1400:
                    if 10 <= V < 50:
                        fc = 544 * V**(-0.63)
                    elif 50 <= V < 130:
                        fc = 85 - 1.108*V + 0.0077*V**2
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 50:
                        fc = 879 * V**(-0.72)
                    elif 50 <= V < 130:
                        fc = 71 - 0.7032*V + 0.0059*V**2
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 50:
                        fc = 1224 * V**(-0.756)
                    elif 50 <= V < 130:
                        fc = 111 - 1.333*V + 0.0093*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            elif 1986 <= yr <= 1992:
                if cc < 1400:
                    if 10 <= V < 17.9:
                        fc = 296.7 - 80.21*ln(V)
                    elif 17.9 <= V < 130:
                        fc = 81.1 - 1.014*V + 0.0068*V**2
                    else:
                        fc = 0
                elif 1400 <= cc < 2000:
                    if 10 <= V < 22.3:
                        fc = 606.1 * V**(-0.667)
                    elif 22.3 <= V < 130:
                        fc = 102.5 - 1.364*V + 0.0086*V**2
                    else:
                        fc = 0
                elif cc >= 2000:
                    if 10 <= V < 59.5:
                        fc = 819.9 * V**(-0.663)
                    elif 59.5 <= V < 130:
                        fc = 41.7 - 0.122*V + 0.0016*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            elif yr > 1993:
                if cc < 1400:
                    if 10 <= V < 130:
                        fc = 85.55 - 1.383*V + 0.0117*V**2
                elif 1400 <= V <2000:
                    if 10 <= V < 130:
                        fc = 109.6 - 1.98*V + 0.0168*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '휘발유' and ty == '경형':
            if twg < 3500:
                if 10 <= V < 110:
                    fc = 0.0167*V**2 - 2.649*V + 161.51
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '휘발유' and ty == '대형':
            if twg > 3500:
                if V <= 60:
                    fc = 225
                elif 60 < V <= 90:
                    fc = 150
                elif V > 90:
                    fc = 165
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '경유' and ty != '경형' and ty != '대형':
            if yr <= 1991:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 91.106 - 1.308*V + 0.00871*V**2
            else:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 118.489 - 2.084*V + 0.014*V**2
                    else:
                        fc = 0
                else:
                    fc = 0
        elif fuel == '경유' and ty == '경형':
            if twg < 3500:
                if 10 <= V < 110:
                    fc = 0.02113*V**2 - 2.65*V + 148.91
                else:
                    fc = 0
            else:
                fc = 0
        elif fuel == '경유' and ty == '대형':
            if twg > 3500:
                if twg < 7500:
                    if 0 <= V < 47:
                        fc = 1425.2*V**(-0.7593)
                    elif 47 <= V < 100:
                        fc = 0.0082*V**2 - 0.0430*V + 60.12
                    else:
                        fc = 0
                elif 7500 <= twg < 16000:
                    if 0 <= V < 59:
                        fc = 1068.4*V**(-0.4905)
                    elif 59 <= V < 100:
                        fc = 0.0126*V**2 - 0.6589*V + 141.18
                    else:
                        fc = 0
                elif 16000 <= twg < 32000:
                    if 0 <= V < 59:
                        fc = 1595.1*V**(-0.4744)
                    elif 59 <= V < 100:
                        fc = 0.0382*V**2 - 5.1630*V + 399.3
                    else:
                        fc = 0
                elif twg > 32000:
                    if 0 <= V < 58:
                        fc = 1855.7*V**(-0.4367)
                    elif 58 <= V < 100:
                        fc = 0.0765*V**2 - 11.414*V + 720.9
                    else:
                        fc = 0
        elif fuel == 'LPG':
            if yr <= 1991:
                if twg < 2500:
                    if 10 <= V < 130:
                        fc = 0.00720*V**2 - 0.9250*V + 74.625
                    else:
                        fc = 0
                else:
                    fc = 0
            else:
                if twg < 2500:
                    if V <= 60:
                        fc = 59
                    elif 60 < V <= 90:
                        fc = 45
                    else:
                        fc = 0
                else:
                    fc = 0
        else:
            fc = 0

        if V < 60:
            fcm += fc*0.34
        elif 60 <= V <= 90:
            fcm += fc*0.33
        elif V > 90:
            fcm += fc*0.33
    fc_list.append(fcm)

100%|██████████| 25117098/25117098 [01:13<00:00, 344013.31it/s]


In [113]:
df2.shape, len(fc_list)

((25117098, 67), 25117098)

In [114]:
df2['FC'] = fc_list

In [115]:
df2[df2['FC'] == 0].shape

(15718235, 68)

In [116]:
r_HC_list = []
for fuel in tqdm(df2['fuel']):
    if fuel == '휘발유':
        r_HC = 1.8
    elif fuel == '경유':
        r_HC = 2.0
    else:
        r_HC = 1.3 # chatGPT 최이사님(2023.06.22.목)
    r_HC_list.append(r_HC)

  0%|          | 0/25117098 [00:00<?, ?it/s]

100%|██████████| 25117098/25117098 [00:13<00:00, 1809426.69it/s]


In [117]:
df2.shape, len(r_HC_list)

((25117098, 68), 25117098)

In [118]:
df2['r_HC'] = r_HC_list

In [119]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리',
       '저감장치부착유무', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC',
       'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5',
       'EFi_NH3', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10',
       'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx',
       'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO', 'E_COLD_NOx',
       'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d', 'e_RHOT', 'e_SHOT',
       'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c', 'E_EVA_VOC',
       'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total', 'E_NH3_total', 'FC',
       'r_HC'],
      dtype='object')

In [120]:
df2['E_CO_total'].describe()

count   25117098.00000
mean           1.79238
std            7.05305
min            0.00020
25%            0.03372
50%            0.60072
75%            0.86260
max         1038.76080
Name: E_CO_total, dtype: float64

In [121]:
df2.loc[df2['E_CO_total'] > 3000, ['E_HOT_CO', 'E_COLD_CO', 'E_EVA_VOC', 'E_CO_total', '차종', '차종유형', 'VKT', '총중량', '배기량', '차량연식']]

,E_HOT_CO,E_COLD_CO,E_EVA_VOC,E_CO_total,차종,차종유형,VKT,총중량,배기량,차량연식


In [122]:
df2['E_CO2_total'] = 44.011 * ( (df2['FC'] / 1000) / (12.011 + 1.008 * df2['r_HC'])) - df2['E_CO_total'] / 28.011 - df2['E_HC_total'] / 13.85 - (df2['E_PM10_total'] + df2['E_PM2_5_total']) / 12.011

In [123]:
df2['E_CO2_total'].describe()

count   25117098.00000
mean          -0.07717
std            0.34411
min          -41.88836
25%           -0.05850
50%           -0.04809
75%            0.00045
max            0.30670
Name: E_CO2_total, dtype: float64

In [124]:
df2[df2['E_CO2_total'] < 0].shape

(18791536, 70)

### 음수 0으로 처리

In [125]:
df2.loc[df2['E_CO2_total'] < 0, 'E_CO2_total'] = 0

## [출력]

In [126]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230630'

In [127]:
# # 24m 55.9s
# df2.to_csv(os.path.join(df_fold, f'CAPSS전체결과({today_date}).csv'), index=False)

In [128]:
# f'CAPSS전체결과({today_date})'

# [로드] 계산 결과

In [129]:
# # 
# date = '20230629'
# name = f'CAPSS전체결과({date})'
# file_name = f'{name}.csv'
# df2 = pd.read_csv(os.path.join(df_fold, file_name), low_memory=False)
# df2.info()

# 통계

## 배출량 통계1
- 조건
    - 승용(경형, 소형, 중형)
    - 화물(소형, 중형)

In [130]:
ad = df2.loc[(df2['차종'] == '승용') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) | (df2['차종'] == '화물') & ((df2['차종유형'] == '소형') | (df2['차종유형'] == '중형'))].reset_index(drop=True)
ad.shape

(17575496, 70)

In [131]:
adg = ad[ad['fuel'] == '휘발유']
adl = ad[ad['fuel'] == 'LPG']
add = ad[ad['fuel'] == '경유']
ad.shape[0] == adg.shape[0] + adl.shape[0] + add.shape[0]

True

In [132]:
grp1 = adg.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,708955.59391,365394.95349,111695.69539,1343.07833,1235.63206,3679.16445,36167.30120,39.67836,988804
2,15869412.10707,3010886.61714,832757.33264,10862.58692,9993.57442,27498.47645,309012.12042,144.88771,7402335
3,3106088.20226,333199.63082,526772.64132,1287.89945,1184.86749,1985.29379,123132.23386,0.00228,542026
4,4838850.16562,706087.60407,1052440.64746,777.25440,715.07405,1076.18663,96565.74915,0.00000,291556
5,3385282.82208,486639.56912,244600.59869,114.11216,104.98319,110.00234,8226.42146,0.00000,28848
X,115824.78451,28874.28036,10801.02509,101.98256,93.82395,256.80015,2972.72836,1.51948,68388


In [133]:
grp1['CO'] = grp1['E_CO_total'] / grp1['차대번호']
grp1['NOx'] = grp1['E_NOx_total'] / grp1['차대번호']
grp1['SOx'] = grp1['E_SOx_total'] / grp1['차대번호']
grp1['PM10'] = grp1['E_PM10_total'] / grp1['차대번호']
grp1['PM2.5'] = grp1['E_PM2_5_total'] / grp1['차대번호']
grp1['HC'] = grp1['E_HC_total'] / grp1['차대번호']
grp1['NH3'] = grp1['E_NH3_total'] / grp1['차대번호']
grp1['CO2'] = grp1['E_CO2_total'] / grp1['차대번호']
grp1

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,708955.59391,365394.95349,111695.69539,1343.07833,1235.63206,3679.16445,36167.30120,39.67836,988804,0.71698,0.11296,0.00372,0.00136,0.00125,0.36953,0.03658,0.00004
2,15869412.10707,3010886.61714,832757.33264,10862.58692,9993.57442,27498.47645,309012.12042,144.88771,7402335,2.14384,0.11250,0.00371,0.00147,0.00135,0.40675,0.04175,0.00002
3,3106088.20226,333199.63082,526772.64132,1287.89945,1184.86749,1985.29379,123132.23386,0.00228,542026,5.73052,0.97186,0.00366,0.00238,0.00219,0.61473,0.22717,0.00000
4,4838850.16562,706087.60407,1052440.64746,777.25440,715.07405,1076.18663,96565.74915,0.00000,291556,16.59664,3.60974,0.00369,0.00267,0.00245,2.42179,0.33121,0.00000
5,3385282.82208,486639.56912,244600.59869,114.11216,104.98319,110.00234,8226.42146,0.00000,28848,117.34896,8.47894,0.00381,0.00396,0.00364,16.86909,0.28516,0.00000
X,115824.78451,28874.28036,10801.02509,101.98256,93.82395,256.80015,2972.72836,1.51948,68388,1.69364,0.15794,0.00376,0.00149,0.00137,0.42221,0.04347,0.00002


In [134]:
grp1_1 = adg.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp1_1

차종,승용,화물
등급,,
1,988804.00000,NaN
2,7382300.00000,20035.00000
3,541308.00000,718.00000
4,288876.00000,2680.00000
5,26955.00000,1893.00000
X,68120.00000,268.00000


In [174]:
grp2 = adl.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp2

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,681707.56,14986.29,138690.34,1962.06,1805.10,88003.06,86021.36,82913.68,476093
2,1424469.70,30309.57,270141.97,3510.40,3229.57,138687.46,136220.75,121022.38,740270
3,4870979.72,127225.21,925043.20,1400.11,1288.10,36700.05,35916.66,34704.56,201291
4,825142.92,51050.55,306327.65,9614.82,8919.38,2815.43,2761.52,2111.01,14390
5,49191.42,5340.57,7157.20,3.49,3.21,78.74,76.97,69.46,428
X,10729.17,241.80,2056.05,28.33,26.06,1276.98,1248.57,1105.52,6690


In [175]:
grp2['CO'] = grp2['E_CO_total'] / grp2['차대번호']
grp2['NOx'] = grp2['E_NOx_total'] / grp2['차대번호']
grp2['SOx'] = grp2['E_SOx_total'] / grp2['차대번호']
grp2['PM10'] = grp2['E_PM10_total'] / grp2['차대번호']
grp2['PM2.5'] = grp2['E_PM2_5_total'] / grp2['차대번호']
grp2['HC'] = grp2['E_HC_total'] / grp2['차대번호']
grp2['NH3'] = grp2['E_NH3_total'] / grp2['차대번호']
grp2['CO2'] = grp2['E_CO2_total'] / grp2['차대번호']
grp2

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,681707.56,14986.29,138690.34,1962.06,1805.10,88003.06,86021.36,82913.68,476093,1.43,0.29,0.18,0.00,0.00,0.03,0.18,0.17
2,1424469.70,30309.57,270141.97,3510.40,3229.57,138687.46,136220.75,121022.38,740270,1.92,0.36,0.19,0.00,0.00,0.04,0.18,0.16
3,4870979.72,127225.21,925043.20,1400.11,1288.10,36700.05,35916.66,34704.56,201291,24.20,4.60,0.18,0.01,0.01,0.63,0.18,0.17
4,825142.92,51050.55,306327.65,9614.82,8919.38,2815.43,2761.52,2111.01,14390,57.34,21.29,0.20,0.67,0.62,3.55,0.19,0.15
5,49191.42,5340.57,7157.20,3.49,3.21,78.74,76.97,69.46,428,114.93,16.72,0.18,0.01,0.01,12.48,0.18,0.16
X,10729.17,241.80,2056.05,28.33,26.06,1276.98,1248.57,1105.52,6690,1.60,0.31,0.19,0.00,0.00,0.04,0.19,0.17


In [176]:
grp2_1 = adl.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp2_1

차종,승용,화물
등급,,
1,476093.00,NaN
2,695585.00,44685.00
3,200734.00,557.00
4,11670.00,2720.00
5,423.00,5.00
X,6348.00,342.00


In [177]:
grp3 = add.groupby(['등급', '저감장치부착유무'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp3

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  유          328648.84   243116.60  54511823.43      70671.79       64926.37  \
4  유               0.58        2.29       994.60          1.47           1.34   
   NaN        824810.53   272087.92  22430903.77     468154.42      430319.56   
5  유           10705.04    40753.91   4523895.79      28589.85       26801.73   
   NaN      14448574.30  3354099.77  31174567.70    1766112.04     1639439.84   
X  유              11.49        6.22      1881.68         20.47          20.39   
   NaN         73857.56     5972.95    122838.98       2721.13        2535.16   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호  
등급 저감장치부착유무                                                  
3  유          1194307.60    118239.56    535420.31  5213000  
4  유               10.90         0.28         0.50       15  
   NaN         207573.09     13566.04     58250.42   835890  
5  유            49669.36      1241.34      3177.32    72050  
   NaN         315377.72     11202.53     30490.02   673947  
X  유               14.94         1.12        13.05       82  
   NaN           4937.90       404.14      1181.95    19393

In [178]:
grp3['CO'] = grp3['E_CO_total'] / grp3['차대번호']
grp3['NOx'] = grp3['E_NOx_total'] / grp3['차대번호']
grp3['SOx'] = grp3['E_SOx_total'] / grp3['차대번호']
grp3['PM10'] = grp3['E_PM10_total'] / grp3['차대번호']
grp3['PM2.5'] = grp3['E_PM2_5_total'] / grp3['차대번호']
grp3['HC'] = grp3['E_HC_total'] / grp3['차대번호']
grp3['NH3'] = grp3['E_NH3_total'] / grp3['차대번호']
grp3['CO2'] = grp3['E_CO2_total'] / grp3['차대번호']
grp3

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  유          328648.84   243116.60  54511823.43      70671.79       64926.37  \
4  유               0.58        2.29       994.60          1.47           1.34   
   NaN        824810.53   272087.92  22430903.77     468154.42      430319.56   
5  유           10705.04    40753.91   4523895.79      28589.85       26801.73   
   NaN      14448574.30  3354099.77  31174567.70    1766112.04     1639439.84   
X  유              11.49        6.22      1881.68         20.47          20.39   
   NaN         73857.56     5972.95    122838.98       2721.13        2535.16   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호    CO   NOx  SOx   
등급 저감장치부착유무                                                                    
3  유          1194307.60    118239.56    535420.31  5213000  0.06 10.46 0.23  \
4  유               10.90         0.28         0.50       15  0.04 66.31 0.73   
   NaN         207573.09     13566.04     58250.42   835890  0.99 26.83 0.25   
5  유            49669.36      1241.34      3177.32    72050  0.15 62.79 0.69   
   NaN         315377.72     11202.53     30490.02   673947 21.44 46.26 0.47   
X  유               14.94         1.12        13.05       82  0.14 22.95 0.18   
   NaN           4937.90       404.14      1181.95    19393  3.81  6.33 0.25   

             PM10  PM2.5   HC  NH3  CO2  
등급 저감장치부착유무                              
3  유         0.01   0.01 0.05 0.02 0.10  
4  유         0.10   0.09 0.15 0.02 0.03  
   NaN       0.56   0.51 0.33 0.02 0.07  
5  유         0.40   0.37 0.57 0.02 0.04  
   NaN       2.62   2.43 4.98 0.02 0.05  
X  유         0.25   0.25 0.08 0.01 0.16  
   NaN       0.14   0.13 0.31 0.02 0.06

In [179]:
grp3_1 = add.groupby(['등급', '저감장치부착유무', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp3_1

차종                승용       화물
등급 저감장치부착유무                  
3  유         3222270  1990730
4  유               3       12
   NaN        352168   483722
5  유           21827    50223
   NaN        131619   542328
X  유              81        1
   NaN          7277    12116

In [153]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.28_AM.11.10.00'

In [180]:
with pd.ExcelWriter(os.path.join(an_fold, f'통계_연료_등급별_배출가스배출량({today_date}).xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='휘발유')
    grp1_1.to_excel(writer, sheet_name='휘발유_차종')
    grp2.to_excel(writer, sheet_name='LPG')
    grp2_1.to_excel(writer, sheet_name='LPG_차종')
    grp3.to_excel(writer, sheet_name='경유')
    grp3_1.to_excel(writer, sheet_name='경유_차종')

## 배출량 통계2
- 조건
    - 승용(대형)
    - 화물(대형)

In [155]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '승용') & (df2['차종유형'] == '대형'), '등급'].value_counts(dropna=False)

등급
3    1744356
5     175072
4     151014
X       8308
Name: count, dtype: int64

In [156]:
df2.loc[(df2['fuel'] == '경유') & (df2['차종'] == '화물') & (df2['차종유형'] == '대형'), '등급'].value_counts(dropna=False)

등급
3    189680
5     86825
4     21367
X      1080
Name: count, dtype: int64

In [157]:
ad2 = df2.loc[((df2['차종'] == '승용') & (df2['차종유형'] == '대형')) | ((df2['차종'] == '화물') & (df2['차종유형'] == '대형'))].reset_index(drop=True)
ad2.shape

(6662922, 70)

In [181]:
ad2g = ad2[ad2['fuel'] == '휘발유']
ad2l = ad2[ad2['fuel'] == 'LPG']
ad2d = ad2[ad2['fuel'] == '경유']
ad2.shape[0] == ad2g.shape[0] + ad2l.shape[0] + ad2d.shape[0]

True

In [182]:
grp21 = ad2g.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp21

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
1,2852082.90,394179.39,267681.55,5656.45,5203.93,12602.31,140918.11,0.00,911275
2,8560435.52,1133516.68,815249.17,17179.62,15805.25,35568.97,430460.50,953.89,2604532
3,4221979.48,369944.18,759005.97,3685.81,3390.94,4793.83,203379.10,38.45,355980
4,2123898.64,270987.00,440586.91,421.80,388.06,476.27,37865.29,3.43,35399
5,369178.17,45789.49,25736.12,13.23,12.18,15.04,1168.65,9.10,1106
X,193985.71,24611.15,17226.08,202.64,186.43,436.07,5624.72,14.33,31661


In [183]:
grp21['CO'] = grp21['E_CO_total'] / grp21['차대번호']
grp21['NOx'] = grp21['E_NOx_total'] / grp21['차대번호']
grp21['SOx'] = grp21['E_SOx_total'] / grp21['차대번호']
grp21['PM10'] = grp21['E_PM10_total'] / grp21['차대번호']
grp21['PM2.5'] = grp21['E_PM2_5_total'] / grp21['차대번호']
grp21['HC'] = grp21['E_HC_total'] / grp21['차대번호']
grp21['NH3'] = grp21['E_NH3_total'] / grp21['차대번호']
grp21['CO2'] = grp21['E_CO2_total'] / grp21['차대번호']
grp21

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
1,2852082.90,394179.39,267681.55,5656.45,5203.93,12602.31,140918.11,0.00,911275,3.13,0.29,0.01,0.01,0.01,0.43,0.15,0.00
2,8560435.52,1133516.68,815249.17,17179.62,15805.25,35568.97,430460.50,953.89,2604532,3.29,0.31,0.01,0.01,0.01,0.44,0.17,0.00
3,4221979.48,369944.18,759005.97,3685.81,3390.94,4793.83,203379.10,38.45,355980,11.86,2.13,0.01,0.01,0.01,1.04,0.57,0.00
4,2123898.64,270987.00,440586.91,421.80,388.06,476.27,37865.29,3.43,35399,60.00,12.45,0.01,0.01,0.01,7.66,1.07,0.00
5,369178.17,45789.49,25736.12,13.23,12.18,15.04,1168.65,9.10,1106,333.80,23.27,0.01,0.01,0.01,41.40,1.06,0.01
X,193985.71,24611.15,17226.08,202.64,186.43,436.07,5624.72,14.33,31661,6.13,0.54,0.01,0.01,0.01,0.78,0.18,0.00


In [184]:
grp21_1 = ad2g.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp21_1

차종,승용,화물
등급,,
1,911275.00,NaN
2,2604522.00,10.00
3,355976.00,4.00
4,35395.00,4.00
5,1092.00,14.00
X,31637.00,24.00


In [185]:
grp22 = ad2l.groupby(['등급'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp22

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호
등급,,,,,,,,,
2,546296.82,11790.98,107141.26,1412.93,1299.89,58843.43,57518.36,54088.99,314737
3,712287.37,16867.69,124329.98,204.62,188.25,5290.97,5171.82,4959.20,29005
4,49150.86,2212.02,9183.74,11.16,10.94,158.58,155.09,102.12,871
5,220.27,25.51,35.39,0.02,0.02,0.42,0.38,0.16,2
X,992.17,21.31,194.72,2.69,2.47,120.63,117.91,70.53,652


In [186]:
grp22['CO'] = grp22['E_CO_total'] / grp22['차대번호']
grp22['NOx'] = grp22['E_NOx_total'] / grp22['차대번호']
grp22['SOx'] = grp22['E_SOx_total'] / grp22['차대번호']
grp22['PM10'] = grp22['E_PM10_total'] / grp22['차대번호']
grp22['PM2.5'] = grp22['E_PM2_5_total'] / grp22['차대번호']
grp22['HC'] = grp22['E_HC_total'] / grp22['차대번호']
grp22['NH3'] = grp22['E_NH3_total'] / grp22['차대번호']
grp22['CO2'] = grp22['E_CO2_total'] / grp22['차대번호']
grp22

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차대번호,CO,NOx,SOx,PM10,PM2.5,HC,NH3,CO2
등급,,,,,,,,,,,,,,,,,
2,546296.82,11790.98,107141.26,1412.93,1299.89,58843.43,57518.36,54088.99,314737,1.74,0.34,0.19,0.00,0.00,0.04,0.18,0.17
3,712287.37,16867.69,124329.98,204.62,188.25,5290.97,5171.82,4959.20,29005,24.56,4.29,0.18,0.01,0.01,0.58,0.18,0.17
4,49150.86,2212.02,9183.74,11.16,10.94,158.58,155.09,102.12,871,56.43,10.54,0.18,0.01,0.01,2.54,0.18,0.12
5,220.27,25.51,35.39,0.02,0.02,0.42,0.38,0.16,2,110.13,17.70,0.21,0.01,0.01,12.76,0.19,0.08
X,992.17,21.31,194.72,2.69,2.47,120.63,117.91,70.53,652,1.52,0.30,0.19,0.00,0.00,0.03,0.18,0.11


In [187]:
grp22_1 = ad2l.groupby(['등급', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp22_1

차종,승용,화물
등급,,
2,314737.00,NaN
3,29005.00,NaN
4,870.00,1.00
5,1.00,1.00
X,652.00,NaN


In [188]:
grp23 = ad2d.groupby(['등급', '저감장치부착유무'], dropna=False).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum', '차대번호':'count'})
grp23

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  유          204383.35   103759.96  24653074.26      26410.32       24297.56  \
4  유               1.04        2.90       949.28          2.14           1.97   
   NaN        508190.38   100432.37   5793278.03      47529.95       43728.18   
5  유           15858.04    63750.58  10160466.26      87249.40       81107.82   
   NaN       4942502.79  1396594.28  24351985.06    1262250.56     1176417.39   
X  유              11.40        9.66      2278.08         23.41          22.98   
   NaN         35484.36     5997.33    150093.66       4594.57        4280.72   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호  
등급 저감장치부착유무                                                  
3  유           635855.24     75192.38    106448.94  1934036  
4  유                9.99         0.38         2.30       12  
   NaN          62812.89      5483.51     12565.43   172369  
5  유            51858.35      1922.07     12894.22    57050  
   NaN         122264.64      5138.62     28715.44   204847  
X  유               15.02         1.01         1.44       69  
   NaN           3302.86       338.92       550.86     9319

In [189]:
grp23['CO'] = grp23['E_CO_total'] / grp23['차대번호']
grp23['NOx'] = grp23['E_NOx_total'] / grp23['차대번호']
grp23['SOx'] = grp23['E_SOx_total'] / grp23['차대번호']
grp23['PM10'] = grp23['E_PM10_total'] / grp23['차대번호']
grp23['PM2.5'] = grp23['E_PM2_5_total'] / grp23['차대번호']
grp23['HC'] = grp23['E_HC_total'] / grp23['차대번호']
grp23['NH3'] = grp23['E_NH3_total'] / grp23['차대번호']
grp23['CO2'] = grp23['E_CO2_total'] / grp23['차대번호']
grp23

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  E_PM2_5_total   
등급 저감장치부착유무                                                                     
3  유          204383.35   103759.96  24653074.26      26410.32       24297.56  \
4  유               1.04        2.90       949.28          2.14           1.97   
   NaN        508190.38   100432.37   5793278.03      47529.95       43728.18   
5  유           15858.04    63750.58  10160466.26      87249.40       81107.82   
   NaN       4942502.79  1396594.28  24351985.06    1262250.56     1176417.39   
X  유              11.40        9.66      2278.08         23.41          22.98   
   NaN         35484.36     5997.33    150093.66       4594.57        4280.72   

             E_SOx_total  E_NH3_total  E_CO2_total     차대번호    CO    NOx  SOx   
등급 저감장치부착유무                                                                     
3  유           635855.24     75192.38    106448.94  1934036  0.11  12.75 0.33  \
4  유                9.99         0.38         2.30       12  0.09  79.11 0.83   
   NaN          62812.89      5483.51     12565.43   172369  2.95  33.61 0.36   
5  유            51858.35      1922.07     12894.22    57050  0.28 178.10 0.91   
   NaN         122264.64      5138.62     28715.44   204847 24.13 118.88 0.60   
X  유               15.02         1.01         1.44       69  0.17  33.02 0.22   
   NaN           3302.86       338.92       550.86     9319  3.81  16.11 0.35   

             PM10  PM2.5   HC  NH3  CO2  
등급 저감장치부착유무                              
3  유         0.01   0.01 0.05 0.04 0.06  
4  유         0.18   0.16 0.24 0.03 0.19  
   NaN       0.28   0.25 0.58 0.03 0.07  
5  유         1.53   1.42 1.12 0.03 0.23  
   NaN       6.16   5.74 6.82 0.03 0.14  
X  유         0.34   0.33 0.14 0.01 0.02  
   NaN       0.49   0.46 0.64 0.04 0.06

In [190]:
grp23_1 = ad2d.groupby(['등급', '저감장치부착유무', '차종'], dropna=False)['차대번호'].count().unstack('차종')
grp23_1

차종                승용      화물
등급 저감장치부착유무                 
3  유         1744356  189680
4  유               7       5
   NaN        151007   21362
5  유           30192   26858
   NaN        144880   59967
X  유              66       3
   NaN          8242    1077

In [193]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.28_AM.11.28.00'

In [194]:
with pd.ExcelWriter(os.path.join(an_fold, f'통계_연료_등급별_배출가스배출량_대형({today_date}).xlsx')) as writer:
    grp21.to_excel(writer, sheet_name='휘발유')
    grp21_1.to_excel(writer, sheet_name='휘발유_차종')
    grp22.to_excel(writer, sheet_name='LPG')
    grp22_1.to_excel(writer, sheet_name='LPG_차종')
    grp23.to_excel(writer, sheet_name='경유')
    grp23_1.to_excel(writer, sheet_name='경유_차종')

## 배출량 통계3

In [135]:
# 1m 0.5s
csm = df2.loc[( (df2['차종'] == '승용') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) ) | 
             ( (df2['차종'] == '화물') & ((df2['차종유형'] == '소형') | (df2['차종유형'] == '중형')) )].reset_index(drop=True)
csm.shape

(17575496, 70)

In [136]:
# 9.3s
csmg = csm[csm['fuel'] == '휘발유']
csml = csm[csm['fuel'] == 'LPG']
csmd = csm[csm['fuel'] == '경유']
csm.shape[0] == csmg.shape[0] + csml.shape[0] + csmd.shape[0]

True

In [137]:
# 13.2s
large = df2.loc[((df2['차종'] == '승용') & (df2['차종유형'] == '대형')) | ((df2['차종'] == '화물') & (df2['차종유형'] == '대형'))].reset_index(drop=True)
large.shape

(6662922, 70)

In [138]:
# 3.3s
largeg = large[large['fuel'] == '휘발유']
largel = large[large['fuel'] == 'LPG']
larged = large[large['fuel'] == '경유']
large.shape[0] == largeg.shape[0] + largel.shape[0] + larged.shape[0]

True

### 경소중형

#### 휘발유

In [139]:
fuel = '휘발유'
div = 'HOT'
stat3_csmg_hot = csmg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmg_hot = stat3_csmg_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_hot[f'E_{div}_CO_per'] = stat3_csmg_hot[f'E_{div}_CO'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_HC_per'] = stat3_csmg_hot[f'E_{div}_HC'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_NOx_per'] = stat3_csmg_hot[f'E_{div}_NOx'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_PM10_per'] = stat3_csmg_hot[f'E_{div}_PM10'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot[f'E_{div}_PM2_5_per'] = stat3_csmg_hot[f'E_{div}_PM2_5'] / stat3_csmg_hot['차량대수']
stat3_csmg_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,1,244614.05104,4265.35835,65354.61992,1343.07833,1235.63206,988804,13623.33933,0.24738,0.00431,0.06609,0.00136,0.00125
1,2,5845142.43452,114971.57561,514529.22588,10862.58692,9993.57442,7402335,13579.61050,0.78963,0.01553,0.06951,0.00147,0.00135
2,3,1694495.52038,68448.86865,397384.01968,1287.89945,1184.86749,542026,13400.23905,3.12623,0.12628,0.73315,0.00238,0.00219
3,4,2751892.51121,238052.15956,820321.71443,777.25440,715.07405,291556,13439.72625,9.43864,0.81649,2.81360,0.00267,0.00245
4,5,2783057.38498,353723.80717,242463.90738,114.11216,104.98319,28848,13453.71806,96.47315,12.26164,8.40488,0.00396,0.00364
5,X,52475.66893,2432.30675,7377.84120,101.98256,93.82395,68388,13715.54001,0.76732,0.03557,0.10788,0.00149,0.00137


In [140]:
fuel = '휘발유'
div = 'COLD'
stat3_csmg_cold = csmg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmg_cold = stat3_csmg_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_cold[f'E_{div}_CO_per'] = stat3_csmg_cold[f'E_{div}_CO'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_HC_per'] = stat3_csmg_cold[f'E_{div}_HC'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_NOx_per'] = stat3_csmg_cold[f'E_{div}_NOx'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_PM10_per'] = stat3_csmg_cold[f'E_{div}_PM10'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold[f'E_{div}_PM2_5_per'] = stat3_csmg_cold[f'E_{div}_PM2_5'] / stat3_csmg_cold['차량대수']
stat3_csmg_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,1,464341.54287,12696.87773,46341.07547,0.00000,0.00000,988804,13623.33933,0.46960,0.01284,0.04687,0.00000,0.00000
1,2,10024269.67255,289050.08312,318228.10677,0.00000,0.00000,7402335,13579.61050,1.35420,0.03905,0.04299,0.00000,0.00000
2,3,1411592.68188,74324.37224,129388.62164,0.00000,0.00000,542026,13400.23905,2.60429,0.13712,0.23871,0.00000,0.00000
3,4,2086957.65442,233110.44445,232118.93303,0.00000,0.00000,291556,13439.72625,7.15800,0.79954,0.79614,0.00000,0.00000
4,5,602225.43710,51019.92042,2136.69131,0.00000,0.00000,28848,13453.71806,20.87581,1.76858,0.07407,0.00000,0.00000
5,X,63349.11558,2040.34336,3423.18389,0.00000,0.00000,68388,13715.54001,0.92632,0.02983,0.05006,0.00000,0.00000


In [141]:
fuel = '휘발유'
div = 'EVA'
stat3_csmg_eva = csmg.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_csmg_eva = stat3_csmg_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_eva[f'E_{div}_VOC_per'] = stat3_csmg_eva[f'E_{div}_VOC'] / stat3_csmg_eva['차량대수']
stat3_csmg_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,1,348432.71741,988804,13623.33933,0.35238
1,2,2606864.95841,7402335,13579.61050,0.35217
2,3,190426.38994,542026,13400.23905,0.35132
3,4,234925.00006,291556,13439.72625,0.80576
4,5,81895.84152,28848,13453.71806,2.83887
5,X,24401.63025,68388,13715.54001,0.35681


In [142]:
fuel = '휘발유'
div = 'total'
stat3_csmg_total = csmg.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmg_total = stat3_csmg_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmg_total[f'E_CO_{div}_per'] = stat3_csmg_total[f'E_CO_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_HC_{div}_per'] = stat3_csmg_total[f'E_HC_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_NOx_{div}_per'] = stat3_csmg_total[f'E_NOx_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_PM10_{div}_per'] = stat3_csmg_total[f'E_PM10_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_PM2_5_{div}_per'] = stat3_csmg_total[f'E_PM2_5_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_SOx_{div}_per'] = stat3_csmg_total[f'E_SOx_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_NH3_{div}_per'] = stat3_csmg_total[f'E_NH3_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total[f'E_CO2_{div}_per'] = stat3_csmg_total[f'E_CO2_{div}'] / stat3_csmg_total['차량대수']
stat3_csmg_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,1,708955.59391,365394.95349,111695.69539,1343.07833,1235.63206,3679.16445,36167.30120,39.67836,988804,13623.33933,0.71698,0.36953,0.11296,0.00136,0.00125,0.00372,0.03658,0.00004
1,2,15869412.10707,3010886.61714,832757.33264,10862.58692,9993.57442,27498.47645,309012.12042,144.88771,7402335,13579.61050,2.14384,0.40675,0.11250,0.00147,0.00135,0.00371,0.04175,0.00002
2,3,3106088.20226,333199.63082,526772.64132,1287.89945,1184.86749,1985.29379,123132.23386,0.00228,542026,13400.23905,5.73052,0.61473,0.97186,0.00238,0.00219,0.00366,0.22717,0.00000
3,4,4838850.16562,706087.60407,1052440.64746,777.25440,715.07405,1076.18663,96565.74915,0.00000,291556,13439.72625,16.59664,2.42179,3.60974,0.00267,0.00245,0.00369,0.33121,0.00000
4,5,3385282.82208,486639.56912,244600.59869,114.11216,104.98319,110.00234,8226.42146,0.00000,28848,13453.71806,117.34896,16.86909,8.47894,0.00396,0.00364,0.00381,0.28516,0.00000
5,X,115824.78451,28874.28036,10801.02509,101.98256,93.82395,256.80015,2972.72836,1.51948,68388,13715.54001,1.69364,0.42221,0.15794,0.00149,0.00137,0.00376,0.04347,0.00002


#### LPG

In [143]:
fuel = 'LPG'
div = 'HOT'
stat3_csml_hot = csml.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csml_hot = stat3_csml_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_hot[f'E_{div}_CO_per'] = stat3_csml_hot[f'E_{div}_CO'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_HC_per'] = stat3_csml_hot[f'E_{div}_HC'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_NOx_per'] = stat3_csml_hot[f'E_{div}_NOx'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_PM10_per'] = stat3_csml_hot[f'E_{div}_PM10'] / stat3_csml_hot['차량대수']
stat3_csml_hot[f'E_{div}_PM2_5_per'] = stat3_csml_hot[f'E_{div}_PM2_5'] / stat3_csml_hot['차량대수']
stat3_csml_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,1,53477.23851,1993.75074,52287.77992,433.09733,398.44955,476093,13692.16695,0.11233,0.00419,0.10983,0.00091,0.00084
1,2,125882.01737,4181.52959,99035.94236,774.87102,712.88134,740270,13877.56522,0.17005,0.00565,0.13378,0.00105,0.00096
2,3,1025567.10242,25426.96446,248156.53889,310.17456,285.36059,201291,13505.43247,5.09495,0.12632,1.23282,0.00154,0.00142
3,4,206315.98351,14411.73515,99246.04429,3650.01187,3372.77000,14390,14492.70674,14.33746,1.00151,6.89688,0.25365,0.23438
4,5,11069.25114,1216.54655,2004.97227,0.76995,0.70835,428,13628.40070,25.86274,2.84240,4.68451,0.00180,0.00166
5,X,896.69776,34.44898,756.32455,6.25308,5.75283,6690,14139.16704,0.13404,0.00515,0.11305,0.00093,0.00086


In [144]:
fuel = 'LPG'
div = 'COLD'
stat3_csml_cold = csml.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csml_cold = stat3_csml_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_cold[f'E_{div}_CO_per'] = stat3_csml_cold[f'E_{div}_CO'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_HC_per'] = stat3_csml_cold[f'E_{div}_HC'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_NOx_per'] = stat3_csml_cold[f'E_{div}_NOx'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_PM10_per'] = stat3_csml_cold[f'E_{div}_PM10'] / stat3_csml_cold['차량대수']
stat3_csml_cold[f'E_{div}_PM2_5_per'] = stat3_csml_cold[f'E_{div}_PM2_5'] / stat3_csml_cold['차량대수']
stat3_csml_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,1,22659.16355,271.38041,0.00000,0.00000,0.00000,476093,13692.16695,0.04759,0.00057,0.00000,0.00000,0.00000
1,2,39666.25023,440.96409,0.00000,0.00000,0.00000,740270,13877.56522,0.05358,0.00060,0.00000,0.00000,0.00000
2,3,237754.62645,1849.47482,0.00000,0.00000,0.00000,201291,13505.43247,1.18115,0.00919,0.00000,0.00000,0.00000
3,4,34039.13200,807.95864,0.00000,0.00000,0.00000,14390,14492.70674,2.36547,0.05615,0.00000,0.00000,0.00000
4,5,2328.46680,82.20782,0.00000,0.00000,0.00000,428,13628.40070,5.44034,0.19207,0.00000,0.00000,0.00000
5,X,316.36381,3.72411,0.00000,0.00000,0.00000,6690,14139.16704,0.04729,0.00056,0.00000,0.00000,0.00000


In [145]:
fuel = 'LPG'
div = 'EVA'
stat3_csml_eva = csml.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_csml_eva = stat3_csml_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_eva[f'E_{div}_VOC_per'] = stat3_csml_eva[f'E_{div}_VOC'] / stat3_csml_eva['차량대수']
stat3_csml_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,1,0.00000,476093,13692.16695,0.00000
1,2,0.00000,740270,13877.56522,0.00000
2,3,0.00000,201291,13505.43247,0.00000
3,4,0.00000,14390,14492.70674,0.00000
4,5,0.00000,428,13628.40070,0.00000
5,X,0.00000,6690,14139.16704,0.00000


In [146]:
fuel = 'LPG'
div = 'total'
stat3_csml_total = csml.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csml_total = stat3_csml_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csml_total[f'E_CO_{div}_per'] = stat3_csml_total[f'E_CO_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_HC_{div}_per'] = stat3_csml_total[f'E_HC_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_NOx_{div}_per'] = stat3_csml_total[f'E_NOx_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_PM10_{div}_per'] = stat3_csml_total[f'E_PM10_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_PM2_5_{div}_per'] = stat3_csml_total[f'E_PM2_5_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_SOx_{div}_per'] = stat3_csml_total[f'E_SOx_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_NH3_{div}_per'] = stat3_csml_total[f'E_NH3_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total[f'E_CO2_{div}_per'] = stat3_csml_total[f'E_CO2_{div}'] / stat3_csml_total['차량대수']
stat3_csml_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,1,76136.40205,2265.13115,52287.77992,433.09733,398.44955,29041.00826,17639.72353,20406.80644,476093,13692.16695,0.15992,0.00476,0.10983,0.00091,0.00084,0.06100,0.03705,0.04286
1,2,165548.26760,4622.49368,99035.94236,774.87102,712.88134,45766.86189,26872.98422,28811.48683,740270,13877.56522,0.22363,0.00624,0.13378,0.00105,0.00096,0.06182,0.03630,0.03892
2,3,1263321.72886,27276.43928,248156.53889,310.17456,285.36059,12111.01554,7387.62564,939.12119,201291,13505.43247,6.27610,0.13551,1.23282,0.00154,0.00142,0.06017,0.03670,0.00467
3,4,240355.11551,15219.69379,99246.04429,3650.01187,3372.77000,929.09047,568.94212,0.00000,14390,14492.70674,16.70293,1.05766,6.89688,0.25365,0.23438,0.06457,0.03954,0.00000
4,5,13397.71795,1298.75437,2004.97227,0.76995,0.70835,25.98582,15.78398,0.00000,428,13628.40070,31.30308,3.03447,4.68451,0.00180,0.00166,0.06071,0.03688,0.00000
5,X,1213.06157,38.17309,756.32455,6.25308,5.75283,421.40303,255.96074,271.47759,6690,14139.16704,0.18132,0.00571,0.11305,0.00093,0.00086,0.06299,0.03826,0.04058


#### 경유

In [147]:
fuel = '경유'
div = 'HOT'
stat3_csmd_hot = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmd_hot = stat3_csmd_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_hot[f'E_{div}_CO_per'] = stat3_csmd_hot[f'E_{div}_CO'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_HC_per'] = stat3_csmd_hot[f'E_{div}_HC'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_NOx_per'] = stat3_csmd_hot[f'E_{div}_NOx'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_PM10_per'] = stat3_csmd_hot[f'E_{div}_PM10'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot[f'E_{div}_PM2_5_per'] = stat3_csmd_hot[f'E_{div}_PM2_5'] / stat3_csmd_hot['차량대수']
stat3_csmd_hot

,등급,저감장치부착유무,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,3,유,10697.03142,26480.91739,14814079.32711,15279.43635,14013.03847,5213000,15321.84552,0.00205,0.00508,2.84176,0.00293,0.00269
1,4,유,0.11464,0.61780,273.72511,0.38315,0.34550,15,18670.96667,0.00764,0.04119,18.24834,0.02554,0.02303
2,4,NaN,211038.02861,66678.61628,6112133.08247,135129.35200,124127.64035,835890,15933.37816,0.25247,0.07977,7.31213,0.16166,0.14850
3,5,유,2159.80680,11469.70785,1252499.44976,7650.07347,7130.06596,72050,17228.87033,0.02998,0.15919,17.38375,0.10618,0.09896
4,5,NaN,4246756.80278,979972.16307,8735692.54425,516139.67864,479422.74319,673947,16622.27337,6.30132,1.45408,12.96199,0.76585,0.71137
5,X,유,0.19767,0.82295,502.76094,3.13732,3.17323,82,13627.40854,0.00241,0.01004,6.13123,0.03826,0.03870
6,X,NaN,19825.06308,1690.82029,36364.48518,745.57615,695.70697,19393,16801.43389,1.02228,0.08719,1.87513,0.03845,0.03587


In [148]:
fuel = '경유'
div = 'COLD'
stat3_csmd_cold = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmd_cold = stat3_csmd_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_cold[f'E_{div}_CO_per'] = stat3_csmd_cold[f'E_{div}_CO'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_HC_per'] = stat3_csmd_cold[f'E_{div}_HC'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_NOx_per'] = stat3_csmd_cold[f'E_{div}_NOx'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_PM10_per'] = stat3_csmd_cold[f'E_{div}_PM10'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold[f'E_{div}_PM2_5_per'] = stat3_csmd_cold[f'E_{div}_PM2_5'] / stat3_csmd_cold['차량대수']
stat3_csmd_cold

,등급,저감장치부착유무,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,3,유,83660.41624,39528.32027,339437.72983,4364.75411,4015.57378,5213000,15321.84552,0.01605,0.00758,0.06511,0.00084,0.00077
1,4,유,0.04103,0.01745,0.53881,0.02795,0.02572,15,18670.96667,0.00274,0.00116,0.03592,0.00186,0.00171
2,4,NaN,5192.33971,2406.07128,65308.16800,3307.35137,3042.79209,835890,15933.37816,0.00621,0.00288,0.07813,0.00396,0.00364
3,5,유,798.46883,212.30768,4007.45888,838.76822,860.65717,72050,17228.87033,0.01108,0.00295,0.05562,0.01164,0.01195
4,5,NaN,5831.79754,1333.03153,25036.79567,6247.16408,6473.21451,673947,16622.27337,0.00865,0.00198,0.03715,0.00927,0.00960
5,X,유,2.78538,0.86590,15.47505,2.90900,2.95353,82,13627.40854,0.03397,0.01056,0.18872,0.03548,0.03602
6,X,NaN,230.93347,58.16976,819.36487,30.33845,29.60121,19393,16801.43389,0.01191,0.00300,0.04225,0.00156,0.00153


In [149]:
fuel = '경유'
div = 'EVA'
stat3_csmd_eva = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_csmd_eva = stat3_csmd_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_eva[f'E_{div}_VOC_per'] = stat3_csmd_eva[f'E_{div}_VOC'] / stat3_csmd_eva['차량대수']
stat3_csmd_eva

,등급,저감장치부착유무,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,3,유,0.00000,5213000,15321.84552,0.00000
1,4,유,0.00000,15,18670.96667,0.00000
2,4,NaN,0.00000,835890,15933.37816,0.00000
3,5,유,0.00000,72050,17228.87033,0.00000
4,5,NaN,0.00000,673947,16622.27337,0.00000
5,X,유,0.00000,82,13627.40854,0.00000
6,X,NaN,0.00000,19393,16801.43389,0.00000


In [150]:
fuel = '경유'
div = 'total'
stat3_csmd_total = csmd.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_csmd_total = stat3_csmd_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_csmd_total[f'E_CO_{div}_per'] = stat3_csmd_total[f'E_CO_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_HC_{div}_per'] = stat3_csmd_total[f'E_HC_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_NOx_{div}_per'] = stat3_csmd_total[f'E_NOx_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_PM10_{div}_per'] = stat3_csmd_total[f'E_PM10_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_PM2_5_{div}_per'] = stat3_csmd_total[f'E_PM2_5_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_SOx_{div}_per'] = stat3_csmd_total[f'E_SOx_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_NH3_{div}_per'] = stat3_csmd_total[f'E_NH3_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total[f'E_CO2_{div}_per'] = stat3_csmd_total[f'E_CO2_{div}'] / stat3_csmd_total['차량대수']
stat3_csmd_total

,등급,저감장치부착유무,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,3,유,94357.44765,66009.23766,15153517.05694,19644.19046,18028.61225,320929.81746,39019.05518,128095.52092,5213000,15321.84552,0.01810,0.01266,2.90687,0.00377,0.00346,0.06156,0.00748,0.02457
1,4,유,0.15567,0.63525,274.26392,0.41110,0.37121,2.97914,0.09242,0.11542,15,18670.96667,0.01038,0.04235,18.28426,0.02741,0.02475,0.19861,0.00616,0.00769
2,4,NaN,216230.36832,69084.68756,6177441.25047,138436.70336,127170.43243,55840.44316,4476.79434,7997.43751,835890,15933.37816,0.25868,0.08265,7.39026,0.16562,0.15214,0.06680,0.00536,0.00957
3,5,유,2958.27563,11682.01553,1256506.90863,8488.84169,7990.72312,13576.56377,409.64224,498.28891,72050,17228.87033,0.04106,0.16214,17.43937,0.11782,0.11091,0.18843,0.00569,0.00692
4,5,NaN,4252588.60032,981305.19460,8760729.33992,522386.84272,485895.95769,85838.50774,3696.83532,789.71286,673947,16622.27337,6.30997,1.45606,12.99914,0.77512,0.72097,0.12737,0.00549,0.00117
5,X,유,2.98305,1.68886,518.23600,6.04632,6.12677,4.00257,0.36876,2.20225,82,13627.40854,0.03638,0.02060,6.31995,0.07374,0.07472,0.04881,0.00450,0.02686
6,X,NaN,20055.99655,1748.99005,37183.85005,775.91460,725.30818,1327.38068,133.36737,194.09098,19393,16801.43389,1.03419,0.09019,1.91739,0.04001,0.03740,0.06845,0.00688,0.01001


#### 출력(경소중형)

In [151]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.30_PM.05.04.27'

In [152]:
with pd.ExcelWriter(os.path.join(an_fold, f'통계_연료_등급별_배출가스배출량_경소중형({today_date}).xlsx')) as writer:
    stat3_csmg_hot.to_excel(writer, sheet_name='휘발유_HOT', index=False)
    stat3_csmg_cold.to_excel(writer, sheet_name='휘발유_COLD', index=False)
    stat3_csmg_eva.to_excel(writer, sheet_name='휘발유_EVA', index=False)
    stat3_csmg_total.to_excel(writer, sheet_name='휘발유_TOTAL', index=False)
    stat3_csml_hot.to_excel(writer, sheet_name='LPG_HOT', index=False)
    stat3_csml_cold.to_excel(writer, sheet_name='LPG_COLD', index=False)
    stat3_csml_eva.to_excel(writer, sheet_name='LPG_EVA', index=False)
    stat3_csml_total.to_excel(writer, sheet_name='LPG_TOTAL', index=False)
    stat3_csmd_hot.to_excel(writer, sheet_name='경유_HOT', index=False)
    stat3_csmd_cold.to_excel(writer, sheet_name='경유_COLD', index=False)
    stat3_csmd_eva.to_excel(writer, sheet_name='경유_EVA', index=False)
    stat3_csmd_total.to_excel(writer, sheet_name='경유_TOTAL', index=False)

### 대형

#### 휘발유

In [153]:
fuel = '휘발유'
div = 'HOT'
stat3_largeg_hot = largeg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largeg_hot = stat3_largeg_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_hot[f'E_{div}_CO_per'] = stat3_largeg_hot[f'E_{div}_CO'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_HC_per'] = stat3_largeg_hot[f'E_{div}_HC'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_NOx_per'] = stat3_largeg_hot[f'E_{div}_NOx'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_PM10_per'] = stat3_largeg_hot[f'E_{div}_PM10'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot[f'E_{div}_PM2_5_per'] = stat3_largeg_hot[f'E_{div}_PM2_5'] / stat3_largeg_hot['차량대수']
stat3_largeg_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,1,273131.28008,3441.57358,61259.61850,1248.58022,1148.69380,911275,13663.71034,0.29972,0.00378,0.06722,0.00137,0.00126
1,2,863138.26691,10950.48074,192532.71976,3792.15798,3488.78534,2604532,13492.98804,0.33140,0.00420,0.07392,0.00146,0.00134
2,3,563368.84129,20677.12941,170679.85373,813.58942,748.50227,355980,13305.21074,1.58259,0.05809,0.47946,0.00229,0.00210
3,4,312434.12038,26231.78508,94377.21854,93.10722,85.65864,35399,13292.08660,8.82607,0.74103,2.66610,0.00263,0.00242
4,5,85029.37450,10813.03530,7879.92964,2.92124,2.68754,1106,13339.72694,76.88009,9.77670,7.12471,0.00264,0.00243
5,X,30352.05368,2623.40957,4339.15233,44.72923,41.15089,31661,13601.52596,0.95866,0.08286,0.13705,0.00141,0.00130


In [154]:
fuel = '휘발유'
div = 'COLD'
stat3_largeg_cold = largeg.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largeg_cold = stat3_largeg_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_cold[f'E_{div}_CO_per'] = stat3_largeg_cold[f'E_{div}_CO'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_HC_per'] = stat3_largeg_cold[f'E_{div}_HC'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_NOx_per'] = stat3_largeg_cold[f'E_{div}_NOx'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_PM10_per'] = stat3_largeg_cold[f'E_{div}_PM10'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold[f'E_{div}_PM2_5_per'] = stat3_largeg_cold[f'E_{div}_PM2_5'] / stat3_largeg_cold['차량대수']
stat3_largeg_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,1,515443.50648,10173.02782,43178.23008,0.00000,0.00000,911275,13663.71034,0.56563,0.01116,0.04738,0.00000,0.00000
1,2,1503946.92709,29744.13681,125417.63820,0.00000,0.00000,2604532,13492.98804,0.57743,0.01142,0.04815,0.00000,0.00000
2,3,576696.14081,32083.58631,63796.93970,0.00000,0.00000,355980,13305.21074,1.62002,0.09013,0.17921,0.00000,0.00000
3,4,232257.11682,23502.12711,26196.66857,0.00000,0.00000,35399,13292.08660,6.56112,0.66392,0.74004,0.00000,0.00000
4,5,18591.21576,1576.25764,74.97278,0.00000,0.00000,1106,13339.72694,16.80942,1.42519,0.06779,0.00000,0.00000
5,X,23501.30043,836.28575,1698.33304,0.00000,0.00000,31661,13601.52596,0.74228,0.02641,0.05364,0.00000,0.00000


In [155]:
fuel = '휘발유'
div = 'EVA'
stat3_largeg_eva = largeg.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_largeg_eva = stat3_largeg_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_eva[f'E_{div}_VOC_per'] = stat3_largeg_eva[f'E_{div}_VOC'] / stat3_largeg_eva['차량대수']
stat3_largeg_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,1,321290.18254,911275,13663.71034,0.35257
1,2,916158.55271,2604532,13492.98804,0.35176
2,3,124899.67508,355980,13305.21074,0.35086
3,4,29886.27473,35399,13292.08660,0.84427
4,5,3119.52973,1106,13339.72694,2.82055
5,X,11609.34232,31661,13601.52596,0.36668


In [156]:
fuel = '휘발유'
div = 'total'
stat3_largeg_total = largeg.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largeg_total = stat3_largeg_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largeg_total[f'E_CO_{div}_per'] = stat3_largeg_total[f'E_CO_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_HC_{div}_per'] = stat3_largeg_total[f'E_HC_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_NOx_{div}_per'] = stat3_largeg_total[f'E_NOx_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_PM10_{div}_per'] = stat3_largeg_total[f'E_PM10_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_PM2_5_{div}_per'] = stat3_largeg_total[f'E_PM2_5_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_SOx_{div}_per'] = stat3_largeg_total[f'E_SOx_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_NH3_{div}_per'] = stat3_largeg_total[f'E_NH3_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total[f'E_CO2_{div}_per'] = stat3_largeg_total[f'E_CO2_{div}'] / stat3_largeg_total['차량대수']
stat3_largeg_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,1,788574.78657,334904.78394,104437.84858,1248.58022,1148.69380,3400.74067,33121.74123,0.00000,911275,13663.71034,0.86535,0.36751,0.11461,0.00137,0.00126,0.00373,0.03635,0.00000
1,2,2367085.19400,956853.17026,317950.35796,3792.15798,3488.78534,9598.29958,104924.58128,163.83240,2604532,13492.98804,0.90883,0.36738,0.12208,0.00146,0.00134,0.00369,0.04029,0.00006
2,3,1140064.98211,177660.39080,234476.79342,813.58942,748.50227,1293.61710,66304.56220,1.34350,355980,13305.21074,3.20261,0.49907,0.65868,0.00229,0.00210,0.00363,0.18626,0.00000
3,4,544691.23721,79620.18693,120573.88711,93.10722,85.65864,128.52009,10949.33391,0.00000,35399,13292.08660,15.38719,2.24922,3.40614,0.00263,0.00242,0.00363,0.30931,0.00000
4,5,103620.59026,15508.82267,7954.90242,2.92124,2.68754,4.05552,337.07134,0.00000,1106,13339.72694,93.68950,14.02244,7.19250,0.00264,0.00243,0.00367,0.30477,0.00000
5,X,53853.35411,15069.03765,6037.48537,44.72923,41.15089,117.66829,1401.03600,1.41520,31661,13601.52596,1.70094,0.47595,0.19069,0.00141,0.00130,0.00372,0.04425,0.00004


#### LPG

In [157]:
fuel = 'LPG'
div = 'HOT'
stat3_largel_hot = largel.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largel_hot = stat3_largel_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_hot[f'E_{div}_CO_per'] = stat3_largel_hot[f'E_{div}_CO'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_HC_per'] = stat3_largel_hot[f'E_{div}_HC'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_NOx_per'] = stat3_largel_hot[f'E_{div}_NOx'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_PM10_per'] = stat3_largel_hot[f'E_{div}_PM10'] / stat3_largel_hot['차량대수']
stat3_largel_hot[f'E_{div}_PM2_5_per'] = stat3_largel_hot[f'E_{div}_PM2_5'] / stat3_largel_hot['차량대수']
stat3_largel_hot

,등급,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,2,49122.75757,1616.27477,39772.65414,311.88377,286.93307,314737,13848.93605,0.15608,0.00514,0.12637,0.00099,0.00091
1,3,146127.57323,3302.95341,33245.15620,45.16588,41.55261,29005,13512.26854,5.03801,0.11388,1.14619,0.00156,0.00143
2,4,11352.58733,512.37830,2427.93340,2.53918,2.46971,871,13493.43398,13.03397,0.58826,2.78752,0.00292,0.00284
3,5,54.75256,6.01919,9.91487,0.00380,0.00350,2,14399.25000,27.37628,3.00960,4.95744,0.00190,0.00175
4,X,84.93406,2.87998,72.58895,0.59280,0.54538,652,13704.57439,0.13027,0.00442,0.11133,0.00091,0.00084


In [158]:
fuel = 'LPG'
div = 'COLD'
stat3_largel_cold = largel.groupby(['등급'], as_index=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largel_cold = stat3_largel_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_cold[f'E_{div}_CO_per'] = stat3_largel_cold[f'E_{div}_CO'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_HC_per'] = stat3_largel_cold[f'E_{div}_HC'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_NOx_per'] = stat3_largel_cold[f'E_{div}_NOx'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_PM10_per'] = stat3_largel_cold[f'E_{div}_PM10'] / stat3_largel_cold['차량대수']
stat3_largel_cold[f'E_{div}_PM2_5_per'] = stat3_largel_cold[f'E_{div}_PM2_5'] / stat3_largel_cold['차량대수']
stat3_largel_cold

,등급,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,2,18081.98738,198.14035,0.00000,0.00000,0.00000,314737,13848.93605,0.05745,0.00063,0.00000,0.00000,0.00000
1,3,34539.47317,244.88440,0.00000,0.00000,0.00000,29005,13512.26854,1.19081,0.00844,0.00000,0.00000,0.00000
2,4,2422.40428,35.11801,0.00000,0.00000,0.00000,871,13493.43398,2.78118,0.04032,0.00000,0.00000,0.00000
3,5,5.24249,0.18514,0.00000,0.00000,0.00000,2,14399.25000,2.62125,0.09257,0.00000,0.00000,0.00000
4,X,34.04728,0.38394,0.00000,0.00000,0.00000,652,13704.57439,0.05222,0.00059,0.00000,0.00000,0.00000


In [159]:
fuel = 'LPG'
div = 'EVA'
stat3_largel_eva = largel.groupby(['등급'], as_index=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_largel_eva = stat3_largel_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_eva[f'E_{div}_VOC_per'] = stat3_largel_eva[f'E_{div}_VOC'] / stat3_largel_eva['차량대수']
stat3_largel_eva

,등급,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,2,0.00000,314737,13848.93605,0.00000
1,3,0.00000,29005,13512.26854,0.00000
2,4,0.00000,871,13493.43398,0.00000
3,5,0.00000,2,14399.25000,0.00000
4,X,0.00000,652,13704.57439,0.00000


In [160]:
fuel = 'LPG'
div = 'total'
stat3_largel_total = largel.groupby(['등급'], as_index=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_largel_total = stat3_largel_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_largel_total[f'E_CO_{div}_per'] = stat3_largel_total[f'E_CO_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_HC_{div}_per'] = stat3_largel_total[f'E_HC_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_NOx_{div}_per'] = stat3_largel_total[f'E_NOx_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_PM10_{div}_per'] = stat3_largel_total[f'E_PM10_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_PM2_5_{div}_per'] = stat3_largel_total[f'E_PM2_5_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_SOx_{div}_per'] = stat3_largel_total[f'E_SOx_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_NH3_{div}_per'] = stat3_largel_total[f'E_NH3_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total[f'E_CO2_{div}_per'] = stat3_largel_total[f'E_CO2_{div}'] / stat3_largel_total['차량대수']
stat3_largel_total

,등급,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,2,67204.74495,1814.41512,39772.65414,311.88377,286.93307,19418.33187,11794.83862,12984.60934,314737,13848.93605,0.21353,0.00576,0.12637,0.00099,0.00091,0.06170,0.03748,0.04126
1,3,180667.04640,3547.83781,33245.15620,45.16588,41.55261,1746.01852,1060.54458,0.07855,29005,13512.26854,6.22882,0.12232,1.14619,0.00156,0.00143,0.06020,0.03656,0.00000
2,4,13774.99161,547.49631,2427.93340,2.53918,2.46971,52.32843,31.80303,0.00000,871,13493.43398,15.81515,0.62858,2.78752,0.00292,0.00284,0.06008,0.03651,0.00000
3,5,59.99505,6.20433,9.91487,0.00380,0.00350,0.12261,0.07793,0.00000,2,14399.25000,29.99752,3.10217,4.95744,0.00190,0.00175,0.06131,0.03896,0.00000
4,X,118.98134,3.26392,72.58895,0.59280,0.54538,39.80713,24.17915,17.15059,652,13704.57439,0.18249,0.00501,0.11133,0.00091,0.00084,0.06105,0.03708,0.02630


#### 경유

In [161]:
fuel = '경유'
div = 'HOT'
stat3_larged_hot = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_larged_hot = stat3_larged_hot.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_hot[f'E_{div}_CO_per'] = stat3_larged_hot[f'E_{div}_CO'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_HC_per'] = stat3_larged_hot[f'E_{div}_HC'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_NOx_per'] = stat3_larged_hot[f'E_{div}_NOx'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_PM10_per'] = stat3_larged_hot[f'E_{div}_PM10'] / stat3_larged_hot['차량대수']
stat3_larged_hot[f'E_{div}_PM2_5_per'] = stat3_larged_hot[f'E_{div}_PM2_5'] / stat3_larged_hot['차량대수']
stat3_larged_hot

,등급,저감장치부착유무,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,차량대수,1년주행거리_평균,E_HOT_CO_per,E_HOT_HC_per,E_HOT_NOx_per,E_HOT_PM10_per,E_HOT_PM2_5_per
0,3,유,8032.00351,11488.88629,6902916.65076,5118.36933,4708.91135,1934036,14224.88439,0.00415,0.00594,3.56918,0.00265,0.00243
1,4,유,0.15177,0.75784,276.49632,0.57029,0.52466,12,15506.41667,0.01265,0.06315,23.04136,0.04752,0.04372
2,4,NaN,143621.88802,26291.34938,1616172.26994,13269.90976,12208.51279,172369,14141.24886,0.83322,0.15253,9.37624,0.07699,0.07083
3,5,유,3130.50387,18086.93596,3039213.95146,24892.75232,23055.97214,57050,15838.80872,0.05487,0.31704,53.27281,0.43633,0.40414
4,5,NaN,1426239.75845,401653.77171,7236206.43872,371645.47703,346656.07707,204847,14776.13553,6.96246,1.96075,35.32493,1.81426,1.69227
5,X,유,0.43568,1.99191,637.71310,4.61439,4.50999,69,13587.52174,0.00631,0.02887,9.24222,0.06688,0.06536
6,X,NaN,10263.52341,1646.21170,44084.68547,1341.55439,1251.04640,9319,14533.63494,1.10135,0.17665,4.73062,0.14396,0.13425


In [162]:
fuel = '경유'
div = 'COLD'
stat3_larged_cold = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_CO':'sum', 
                                                    f'E_{div}_HC':'sum', 
                                                    f'E_{div}_NOx':'sum', 
                                                    f'E_{div}_PM10':'sum',
                                                    f'E_{div}_PM2_5':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_larged_cold = stat3_larged_cold.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_cold[f'E_{div}_CO_per'] = stat3_larged_cold[f'E_{div}_CO'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_HC_per'] = stat3_larged_cold[f'E_{div}_HC'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_NOx_per'] = stat3_larged_cold[f'E_{div}_NOx'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_PM10_per'] = stat3_larged_cold[f'E_{div}_PM10'] / stat3_larged_cold['차량대수']
stat3_larged_cold[f'E_{div}_PM2_5_per'] = stat3_larged_cold[f'E_{div}_PM2_5'] / stat3_larged_cold['차량대수']
stat3_larged_cold

,등급,저감장치부착유무,E_COLD_CO,E_COLD_HC,E_COLD_NOx,E_COLD_PM10,E_COLD_PM2_5,차량대수,1년주행거리_평균,E_COLD_CO_per,E_COLD_HC_per,E_COLD_NOx_per,E_COLD_PM10_per,E_COLD_PM2_5_per
0,3,유,51925.80635,17068.53431,174737.92827,2286.37377,2103.47505,1934036,14224.88439,0.02685,0.00883,0.09035,0.00118,0.00109
1,4,유,0.13296,0.06836,1.25909,0.05954,0.05478,12,15506.41667,0.01108,0.00570,0.10492,0.00496,0.00457
2,4,NaN,2980.14846,1764.03129,26640.11620,1146.61932,1054.92081,172369,14141.24886,0.01729,0.01023,0.15455,0.00665,0.00612
3,5,유,1304.53173,311.97382,5523.36141,1273.10047,1320.69522,57050,15838.80872,0.02287,0.00547,0.09682,0.02232,0.02315
4,5,NaN,6521.26905,1551.81957,27608.14220,6360.08821,6602.86993,204847,14776.13553,0.03183,0.00758,0.13477,0.03105,0.03223
5,X,유,2.55989,0.70271,12.59649,2.31988,2.39015,69,13587.52174,0.03710,0.01018,0.18256,0.03362,0.03464
6,X,NaN,280.79228,65.03456,850.92023,32.74222,32.78804,9319,14533.63494,0.03013,0.00698,0.09131,0.00351,0.00352


In [163]:
fuel = '경유'
div = 'EVA'
stat3_larged_eva = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_{div}_VOC':'sum', 
                                                            '차대번호':'count', 
                                                            'VKT':'mean',
                                                            })
stat3_larged_eva = stat3_larged_eva.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_eva[f'E_{div}_VOC_per'] = stat3_larged_eva[f'E_{div}_VOC'] / stat3_larged_eva['차량대수']
stat3_larged_eva

,등급,저감장치부착유무,E_EVA_VOC,차량대수,1년주행거리_평균,E_EVA_VOC_per
0,3,유,0.00000,1934036,14224.88439,0.00000
1,4,유,0.00000,12,15506.41667,0.00000
2,4,NaN,0.00000,172369,14141.24886,0.00000
3,5,유,0.00000,57050,15838.80872,0.00000
4,5,NaN,0.00000,204847,14776.13553,0.00000
5,X,유,0.00000,69,13587.52174,0.00000
6,X,NaN,0.00000,9319,14533.63494,0.00000


In [174]:
fuel = '경유'
div = 'total'
stat3_larged_total = larged.groupby(['등급', '저감장치부착유무'], as_index=False, dropna=False).agg({f'E_CO_{div}':'sum', 
                                                    f'E_HC_{div}':'sum', 
                                                    f'E_NOx_{div}':'sum', 
                                                    f'E_PM10_{div}':'sum',
                                                    f'E_PM2_5_{div}':'sum', 
                                                    f'E_SOx_{div}':'sum', 
                                                    f'E_NH3_{div}':'sum', 
                                                    f'E_CO2_{div}':'sum', 
                                                    '차대번호':'count', 
                                                    'VKT':'mean',
                                                    })
stat3_larged_total = stat3_larged_total.rename(columns={'차대번호':'차량대수', 'VKT':'1년주행거리_평균'})
stat3_larged_total[f'E_CO_{div}_per'] = stat3_larged_total[f'E_CO_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_HC_{div}_per'] = stat3_larged_total[f'E_HC_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_NOx_{div}_per'] = stat3_larged_total[f'E_NOx_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_PM10_{div}_per'] = stat3_larged_total[f'E_PM10_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_PM2_5_{div}_per'] = stat3_larged_total[f'E_PM2_5_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_SOx_{div}_per'] = stat3_larged_total[f'E_SOx_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_NH3_{div}_per'] = stat3_larged_total[f'E_NH3_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total[f'E_CO2_{div}_per'] = stat3_larged_total[f'E_CO2_{div}'] / stat3_larged_total['차량대수']
stat3_larged_total

,등급,저감장치부착유무,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total,차량대수,1년주행거리_평균,E_CO_total_per,E_HC_total_per,E_NOx_total_per,E_PM10_total_per,E_PM2_5_total_per,E_SOx_total_per,E_NH3_total_per,E_CO2_total_per
0,3,유,59957.80987,28557.42060,7077654.57903,7404.74310,6812.38640,172460.56490,24813.48654,69951.15566,1934036,14224.88439,0.03100,0.01477,3.65953,0.00383,0.00352,0.08917,0.01283,0.03617
1,4,유,0.28472,0.82620,277.75541,0.62983,0.57944,2.73649,0.12488,0.86988,12,15506.41667,0.02373,0.06885,23.14628,0.05249,0.04829,0.22804,0.01041,0.07249
2,4,NaN,146602.03649,28055.38067,1642812.38615,14416.52908,13263.43359,17066.65116,1809.55726,2228.61107,172369,14141.24886,0.85051,0.16276,9.53079,0.08364,0.07695,0.09901,0.01050,0.01293
3,5,유,4435.03560,18398.90979,3044737.31286,26165.85279,24376.66736,14209.55715,634.28297,1766.97597,57050,15838.80872,0.07774,0.32250,53.36963,0.45865,0.42729,0.24907,0.01112,0.03097
4,5,NaN,1432761.02749,403205.59128,7263814.58092,378005.56524,353258.94701,33404.91334,1695.74526,1038.49453,204847,14776.13553,6.99430,1.96833,35.45971,1.84531,1.72450,0.16307,0.00828,0.00507
5,X,유,2.99557,2.69462,650.30958,6.93427,6.90014,4.04194,0.33456,0.66156,69,13587.52174,0.04341,0.03905,9.42478,0.10050,0.10000,0.05858,0.00485,0.00959
6,X,NaN,10544.31570,1711.24626,44935.60570,1374.29661,1283.83444,896.62598,111.84451,115.65575,9319,14533.63494,1.13149,0.18363,4.82193,0.14747,0.13777,0.09621,0.01200,0.01241


#### 출력(대형)

In [175]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.30_PM.05.10.45'

In [176]:
with pd.ExcelWriter(os.path.join(an_fold, f'통계_연료_등급별_배출가스배출량_대형({today_date}).xlsx')) as writer:
    stat3_largeg_hot.to_excel(writer, sheet_name='휘발유_HOT', index=False)
    stat3_largeg_cold.to_excel(writer, sheet_name='휘발유_COLD', index=False)
    stat3_largeg_eva.to_excel(writer, sheet_name='휘발유_EVA', index=False)
    stat3_largeg_total.to_excel(writer, sheet_name='휘발유_TOTAL', index=False)
    stat3_largel_hot.to_excel(writer, sheet_name='LPG_HOT', index=False)
    stat3_largel_cold.to_excel(writer, sheet_name='LPG_COLD', index=False)
    stat3_largel_eva.to_excel(writer, sheet_name='LPG_EVA', index=False)
    stat3_largel_total.to_excel(writer, sheet_name='LPG_TOTAL', index=False)
    stat3_larged_hot.to_excel(writer, sheet_name='경유_HOT', index=False)
    stat3_larged_cold.to_excel(writer, sheet_name='경유_COLD', index=False)
    stat3_larged_eva.to_excel(writer, sheet_name='경유_EVA', index=False)
    stat3_larged_total.to_excel(writer, sheet_name='경유_TOTAL', index=False)

## 총 배출량 통계

In [167]:
df2['fuel'].unique()

array(['휘발유', 'LPG', '경유'], dtype=object)

In [168]:
grp3 = df2.groupby('fuel').agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum', 'E_SOx_total':'sum', 'E_NH3_total':'sum', 'E_CO2_total':'sum'})
grp3

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
fuel,,,,,,,,
LPG,3458438.10978,112815.06909,771426.70880,8361.01422,7722.35087,120475.35137,72279.79914,63433.11434
경유,8379589.42559,2171103.80123,61905395.68523,1336790.47234,1260557.05126,951505.34324,92709.86428,228101.31594
휘발유,33181458.75747,6535136.61475,3614460.89829,20520.11882,18878.50181,49212.64984,795225.68784,356.02755


In [169]:
grp3_sum = grp3.sum().reset_index().T
grp3_sum.columns = grp3_sum.iloc[0, :]
grp3_sum.index = ['index', '합계']
grp3_sum = grp3_sum.iloc[[1]]
grp3_sum

index,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
합계,45019486.29284,8819055.48507,66291283.29232,1365671.60539,1287157.90394,1121193.34445,960215.35126,291890.45783


In [170]:
grp3m = pd.concat([grp3, grp3_sum])
grp3m

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_SOx_total,E_NH3_total,E_CO2_total
LPG,3458438.10978,112815.06909,771426.70880,8361.01422,7722.35087,120475.35137,72279.79914,63433.11434
경유,8379589.42559,2171103.80123,61905395.68523,1336790.47234,1260557.05126,951505.34324,92709.86428,228101.31594
휘발유,33181458.75747,6535136.61475,3614460.89829,20520.11882,18878.50181,49212.64984,795225.68784,356.02755
합계,45019486.29284,8819055.48507,66291283.29232,1365671.60539,1287157.90394,1121193.34445,960215.35126,291890.45783


In [171]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.30_PM.05.05.06'

In [172]:
grp3m.to_excel(os.path.join(an_fold, f'통계_CAPSS_배출량({today_date}).xlsx'))

# [출력] 경유, 휘발유, LPG 차량 배출량

In [ ]:
df2.columns

Index(['법정동코드', '차종', '차대번호', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN', '차종유형',
       '연료', '등급', 'fuel', '차령', '시도', '시구군', '일일평균주행거리', '저감장치부착유무', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO',
       'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC',
       'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_total', 'E_PM2_5_total', 'fcc', 'sf', 'E_SOx_total'],
      dtype='object')

In [ ]:
# 컬럼명 정리
ep = df2[[
    '법정동코드', 
    '시도', 
    '시구군', 
    '차대번호', 
    '차량연식', 
    '배출가스인증번호', 
    '등급', 
    'fuel', 
    '차종', 
    '차종유형', 
    '일일평균주행거리', 
    '저감장치부착유무', 
    'E_CO_total', 
    'E_HC_total', 
    'E_NOx_total',
    'E_PM10_total', 
    'E_PM2_5_total', 
    'E_SOx_total',
    ]]
ep.shape

(25117108, 18)

In [ ]:
ep.columns

Index(['법정동코드', '시도', '시구군', '차대번호', '차량연식', '배출가스인증번호', '등급', 'fuel', '차종',
       '차종유형', '일일평균주행거리', '저감장치부착유무', 'E_CO_total', 'E_HC_total',
       'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total', 'E_SOx_total'],
      dtype='object')

In [ ]:
ep = ep.rename(columns={'등급':'배출가스등급', 'fuel':'연료'})

In [ ]:
# # about
# df2.to_csv(os.path.join(df_fold, '경유_휘발유_LPG차량_배출량.csv'), index=False)

# 4등급 차량 차종별 일일평균주행거리 확인
- 이상 주행거리 제외

## [로드] 일일평균주행거리

In [30]:
# about 
name = '일일평균주행거리_최초등록일자_최근검사일자_기준'
file_name = f'{name}.csv'
run = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2009566 entries, 0 to 2009565
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   차대번호      object 
 1   최초등록일자수정  object 
 2   최근검사일자수정  object 
 3   최근주행거리    int64  
 4   등록일기준검사일  int64  
 5   현재일자      object 
 6   일일평균주행거리  float64
 7   최근검사경과일   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 122.7+ MB


In [32]:
run.shape, len(run['차대번호'].unique())

((2009566, 8), 2009566)

In [31]:
smig3.columns

Index(['법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '배출가스인증번호', '배출가스등급', '차량말소YN',
       '차종유형', '연료', '총중량', '배기량', '등급', 'fuel'],
      dtype='object')

In [34]:
smig3.shape

(25692512, 14)

In [33]:
df = smig3.merge(run[['차대번호', '일일평균주행거리']], on='차대번호', how='left')
df.shape

(25692512, 15)

In [36]:
dfs1 = df[df['일일평균주행거리'] < 1000].reset_index(drop=True)
dfs1.shape

(2009532, 15)

In [38]:
dfs1.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index()

,차종,일일평균주행거리
0,승용,37.11
1,승합,43.40
2,특수,102.08
3,화물,43.81


In [40]:
dfs1.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index().to_excel(os.path.join(an1_fold, '통계_차종별_일일평균주행거리.xlsx'), index=False)

In [44]:
df['등급'].unique()

array(['X', '1', '2', '3', '4', '5'], dtype=object)

In [45]:
dfs2 = df[(df['일일평균주행거리'] < 1000) & (df['등급'] == '4')].reset_index(drop=True)
dfs2.shape

(1192850, 15)

In [46]:
dfs2.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index()

,차종,일일평균주행거리
0,승용,38.29
1,승합,45.29
2,특수,156.36
3,화물,48.21


In [47]:
dfs2.groupby('차종').agg({'일일평균주행거리':'mean'}).reset_index().to_excel(os.path.join(an1_fold, '통계_4등급_차종별_일일평균주행거리.xlsx'), index=False)

# 코드 마지막